In [ ]:
pip install --upgrade torch torchvision opacus


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz
100%|██████████| 170M/170M [00:08<00:00, 20.4MB/s]
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch: 1, Train Loss: 1.5723, Train Accuracy: 41.47%
Epoch: 1, Test Loss: 1.3598, Test Accuracy: 50.90%
End of Epoch 1:
Train Loss: 1.5723, Train Accuracy: 41.47%
Test Loss: 1.3598, Test Accuracy: 50.90%
Epoch: 2, Train Loss: 1.0846, Train Accuracy: 61.08%
Epoch: 2, Test Loss: 1.2472, Test Accuracy: 58.09%
End of Epoch 2:
Train Loss: 1.0846, Train Accuracy: 61.08%
Test Loss: 1.2472, Test Accuracy: 58.09%

*DP*- ADDED.   256, 0.1-->no group normalization

---



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.conv1(x), inplace=False)  # Explicitly ensure inplace=False
        out = F.relu(self.conv2(out), inplace=False)
        out = out + self.shortcut(x)  # Avoid inplace modification
        return out


class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Model, Loss, and Optimizer
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

# Initialize SGD optimizer from Opacus
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,  # Adjust this value based on your privacy needs
    max_grad_norm=1.0,
    epochs= 20
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop
for epoch in range(1, 20):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)

    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}')
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:')

    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()



ADDED GROUP NORMALIZATION.          256, 0.1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)  # Replacing BatchNorm with GroupNorm
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)  # Replacing BatchNorm with GroupNorm
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])  # Replacing BatchNorm with GroupNorm
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

# Initialize SGD optimizer from Opacus
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,  # Adjust this value based on your privacy needs
    max_grad_norm=1.0,
    epochs= 20
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop
for epoch in range(1, 20):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)

    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}')
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:')

    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()



Privacy Budget: Epsilon = 2.1299
End of Epoch 8:
Train Loss: 1.8109, Train Accuracy: 32.17%
Test Loss: 1.7563, Test Accuracy: 32.31%

Increased batch size to 512, 0.1





In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn .functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)  # Replacing BatchNorm with GroupNorm
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)  # Replacing BatchNorm with GroupNorm
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])  # Replacing BatchNorm with GroupNorm
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

# Initialize SGD optimizer from Opacus
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=20,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=0.3,  # Adjust this value based on your privacy needs
    max_grad_norm=1.0,     # Set max gradient norm as required
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop
for epoch in range(1, 20):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()






```
# This is formatted as code
```

BATCH SIZE=256, 0.01

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn .functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)  # Replacing BatchNorm with GroupNorm
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)  # Replacing BatchNorm with GroupNorm
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])  # Replacing BatchNorm with GroupNorm
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

# Initialize SGD optimizer from Opacus
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=20,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=0.3,  # Adjust this value based on your privacy needs
    max_grad_norm=1.0,     # Set max gradient norm as required
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop
for epoch in range(1, 20):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()





ADDED WEIGHT STANDARDIZATION- BATCH SZE: 256, 0.001------>(x,y)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

torch.cuda.empty_cache()

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        # Initialize weight_mean and weight_std with the correct dimensions
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))  # (1, in_channels, 1, 1)
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))   # (1, in_channels, 1, 1)

    def forward(self, x):
        weight = self.conv.weight
        # Normalize the weights
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        # Apply the learned scale and shift
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)



class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out



class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Modify the DataLoader to drop the last batch if it's incomplete
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)


# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

# Initialize SGD optimizer from Opacus
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=0.3,  # Adjust this value based on your privacy needs
    max_grad_norm=1.0,     # Set max gradient norm as required
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop
for epoch in range(1, 50):
    torch.cuda.empty_cache()
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()






SAME PREVIOUS WEIGHT STANDARDIZATION CODE. BATCH SIZE=128, 0.001



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

torch.cuda.empty_cache()

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        # Initialize weight_mean and weight_std with the correct dimensions
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))  # (1, in_channels, 1, 1)
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))   # (1, in_channels, 1, 1)

    def forward(self, x):
        weight = self.conv.weight
        # Normalize the weights
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        # Apply the learned scale and shift
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)



class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out



class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Modify the DataLoader to drop the last batch if it's incomplete
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=True)


# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

# Initialize SGD optimizer from Opacus
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=20,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=0.3,  # Adjust this value based on your privacy needs
    max_grad_norm=1.0,     # Set max gradient norm as required
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop
for epoch in range(1, 20):
    torch.cuda.empty_cache()
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


ADD PARAMETER AVERAGING

ADD PARAMETER AVERAGING- lr=0.1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%\n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%\n')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...\n")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}:\n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}\n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%\n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%\n')


print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


PARAMETER AVERAGING- LR=0.01

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


Epoch: 1, Train Loss: 2.1033, Train Accuracy: 20.98%
Epoch: 1, Test Loss: 1.9654, Test Accuracy: 26.69%
End of Epoch 1:
Privacy Budget: Epsilon = 0.4951
Train Loss: 2.1033, Train Accuracy: 20.98%
Test Loss: 1.9654, Test Accuracy: 26.69%
Epoch: 2, Train Loss: 1.8857, Train Accuracy: 29.57%
Epoch: 2, Test Loss: 1.8192, Test Accuracy: 31.95%
End of Epoch 2:
Privacy Budget: Epsilon = 0.6385
Train Loss: 1.8857, Train Accuracy: 29.57%
Test Loss: 1.8192, Test Accuracy: 31.95%
Epoch: 3, Train Loss: 1.7737, Train Accuracy: 33.35%
Epoch: 3, Test Loss: 1.7448, Test Accuracy: 34.95%
End of Epoch 3:
Privacy Budget: Epsilon = 0.7527
Train Loss: 1.7737, Train Accuracy: 33.35%
Test Loss: 1.7448, Test Accuracy: 34.95%
Epoch: 4, Train Loss: 1.7244, Train Accuracy: 34.61%
Epoch: 4, Test Loss: 1.7017, Test Accuracy: 35.83%
End of Epoch 4:
Privacy Budget: Epsilon = 0.8518
Train Loss: 1.7244, Train Accuracy: 34.61%
Test Loss: 1.7017, Test Accuracy: 35.83%
Epoch: 5, Train Loss: 1.6948, Train Accuracy: 35.63%
Epoch: 5, Test Loss: 1.6853, Test Accuracy: 37.67%
End of Epoch 5:
Privacy Budget: Epsilon = 0.9411
Train Loss: 1.6948, Train Accuracy: 35.63%
Test Loss: 1.6853, Test Accuracy: 37.67%
Epoch: 6, Train Loss: 1.6733, Train Accuracy: 37.48%
Epoch: 6, Test Loss: 1.6444, Test Accuracy: 39.04%
End of Epoch 6:
Privacy Budget: Epsilon = 1.0231
Train Loss: 1.6733, Train Accuracy: 37.48%
Test Loss: 1.6444, Test Accuracy: 39.04%
Epoch: 7, Train Loss: 1.6260, Train Accuracy: 39.44%
Epoch: 7, Test Loss: 1.6314, Test Accuracy: 39.70%
End of Epoch 7:
Privacy Budget: Epsilon = 1.0995
Train Loss: 1.6260, Train Accuracy: 39.44%
Test Loss: 1.6314, Test Accuracy: 39.70%
Epoch: 8, Train Loss: 1.6026, Train Accuracy: 40.60%
Epoch: 8, Test Loss: 1.5973, Test Accuracy: 40.53%
End of Epoch 8:
Privacy Budget: Epsilon = 1.1715
Train Loss: 1.6026, Train Accuracy: 40.60%
Test Loss: 1.5973, Test Accuracy: 40.53%
Epoch: 9, Train Loss: 1.5734, Train Accuracy: 41.50%
Epoch: 9, Test Loss: 1.5747, Test Accuracy: 41.48%
End of Epoch 9:
Privacy Budget: Epsilon = 1.2397
Train Loss: 1.5734, Train Accuracy: 41.50%
Test Loss: 1.5747, Test Accuracy: 41.48%
Epoch: 10, Train Loss: 1.5472, Train Accuracy: 42.68%
Epoch: 10, Test Loss: 1.5493, Test Accuracy: 42.29%
Averaging model parameters...
End of Epoch 10:
Privacy Budget: Epsilon = 1.3049
Train Loss: 1.5472, Train Accuracy: 42.68%
Test Loss: 1.5493, Test Accuracy: 42.29%
Epoch: 11, Train Loss: 1.5383, Train Accuracy: 43.20%
Epoch: 11, Test Loss: 1.5379, Test Accuracy: 43.70%
End of Epoch 11:
Privacy Budget: Epsilon = 1.3673
Train Loss: 1.5383, Train Accuracy: 43.20%
Test Loss: 1.5379, Test Accuracy: 43.70%
Epoch: 12, Train Loss: 1.5294, Train Accuracy: 43.69%
Epoch: 12, Test Loss: 1.5338, Test Accuracy: 43.84%
End of Epoch 12:
Privacy Budget: Epsilon = 1.4274
Train Loss: 1.5294, Train Accuracy: 43.69%
Test Loss: 1.5338, Test Accuracy: 43.84%
Epoch: 13, Train Loss: 1.5154, Train Accuracy: 44.49%
Epoch: 13, Test Loss: 1.5147, Test Accuracy: 44.45%
End of Epoch 13:
Privacy Budget: Epsilon = 1.4854
Train Loss: 1.5154, Train Accuracy: 44.49%
Test Loss: 1.5147, Test Accuracy: 44.45%
Epoch: 14, Train Loss: 1.5034, Train Accuracy: 45.14%
Epoch: 14, Test Loss: 1.5295, Test Accuracy: 44.07%
End of Epoch 14:
Privacy Budget: Epsilon = 1.5415
Train Loss: 1.5034, Train Accuracy: 45.14%
Test Loss: 1.5295, Test Accuracy: 44.07%
Epoch: 15, Train Loss: 1.5005, Train Accuracy: 45.34%
Epoch: 15, Test Loss: 1.4931, Test Accuracy: 45.64%
End of Epoch 15:
Privacy Budget: Epsilon = 1.5959
Train Loss: 1.5005, Train Accuracy: 45.34%
Test Loss: 1.4931, Test Accuracy: 45.64%
Epoch: 16, Train Loss: 1.4879, Train Accuracy: 45.65%
Epoch: 16, Test Loss: 1.4916, Test Accuracy: 45.84%
End of Epoch 16:
Privacy Budget: Epsilon = 1.6489
Train Loss: 1.4879, Train Accuracy: 45.65%
Test Loss: 1.4916, Test Accuracy: 45.84%
Epoch: 17, Train Loss: 1.4846, Train Accuracy: 45.96%
Epoch: 17, Test Loss: 1.4768, Test Accuracy: 45.92%
End of Epoch 17:
Privacy Budget: Epsilon = 1.7004
Train Loss: 1.4846, Train Accuracy: 45.96%
Test Loss: 1.4768, Test Accuracy: 45.92%
Epoch: 18, Train Loss: 1.4795, Train Accuracy: 46.10%
Epoch: 18, Test Loss: 1.4799, Test Accuracy: 46.80%
End of Epoch 18:
Privacy Budget: Epsilon = 1.7507
Train Loss: 1.4795, Train Accuracy: 46.10%
Test Loss: 1.4799, Test Accuracy: 46.80%
Epoch: 19, Train Loss: 1.4763, Train Accuracy: 46.70%
Epoch: 19, Test Loss: 1.4994, Test Accuracy: 46.22%
End of Epoch 19:
Privacy Budget: Epsilon = 1.7998
Train Loss: 1.4763, Train Accuracy: 46.70%
Test Loss: 1.4994, Test Accuracy: 46.22%
Epoch: 20, Train Loss: 1.4775, Train Accuracy: 46.73%
Epoch: 20, Test Loss: 1.4746, Test Accuracy: 46.62%
Averaging model parameters...
End of Epoch 20:
Privacy Budget: Epsilon = 1.8479
Train Loss: 1.4775, Train Accuracy: 46.73%
Test Loss: 1.4746, Test Accuracy: 46.62%
Epoch: 21, Train Loss: 1.4610, Train Accuracy: 47.27%
Epoch: 21, Test Loss: 1.4627, Test Accuracy: 46.97%
End of Epoch 21:
Privacy Budget: Epsilon = 1.8949
Train Loss: 1.4610, Train Accuracy: 47.27%
Test Loss: 1.4627, Test Accuracy: 46.97%
Epoch: 22, Train Loss: 1.4689, Train Accuracy: 47.49%
Epoch: 22, Test Loss: 1.4657, Test Accuracy: 47.36%
End of Epoch 22:
Privacy Budget: Epsilon = 1.9410
Train Loss: 1.4689, Train Accuracy: 47.49%
Test Loss: 1.4657, Test Accuracy: 47.36%
Epoch: 23, Train Loss: 1.4579, Train Accuracy: 48.04%
Epoch: 23, Test Loss: 1.4811, Test Accuracy: 48.05%
End of Epoch 23:
Privacy Budget: Epsilon = 1.9863
Train Loss: 1.4579, Train Accuracy: 48.04%
Test Loss: 1.4811, Test Accuracy: 48.05%
Epoch: 24, Train Loss: 1.4636, Train Accuracy: 47.90%
Epoch: 24, Test Loss: 1.4890, Test Accuracy: 46.90%
End of Epoch 24:
Privacy Budget: Epsilon = 2.0307
Train Loss: 1.4636, Train Accuracy: 47.90%
Test Loss: 1.4890, Test Accuracy: 46.90%
Epoch: 25, Train Loss: 1.4640, Train Accuracy: 48.15%
Epoch: 25, Test Loss: 1.4731, Test Accuracy: 48.25%
End of Epoch 25:
Privacy Budget: Epsilon = 2.0743
Train Loss: 1.4640, Train Accuracy: 48.15%
Test Loss: 1.4731, Test Accuracy: 48.25%
Epoch: 26, Train Loss: 1.4610, Train Accuracy: 48.02%
Epoch: 26, Test Loss: 1.4881, Test Accuracy: 48.18%
End of Epoch 26:
Privacy Budget: Epsilon = 2.1172
Train Loss: 1.4610, Train Accuracy: 48.02%
Test Loss: 1.4881, Test Accuracy: 48.18%
Epoch: 27, Train Loss: 1.4693, Train Accuracy: 48.28%
Epoch: 27, Test Loss: 1.4799, Test Accuracy: 48.19%
End of Epoch 27:
Privacy Budget: Epsilon = 2.1594
Train Loss: 1.4693, Train Accuracy: 48.28%
Test Loss: 1.4799, Test Accuracy: 48.19%
Epoch: 28, Train Loss: 1.4669, Train Accuracy: 48.59%
Epoch: 28, Test Loss: 1.4754, Test Accuracy: 48.41%
End of Epoch 28:
Privacy Budget: Epsilon = 2.2010
Train Loss: 1.4669, Train Accuracy: 48.59%
Test Loss: 1.4754, Test Accuracy: 48.41%
Epoch: 29, Train Loss: 1.4600, Train Accuracy: 48.72%
Epoch: 29, Test Loss: 1.4697, Test Accuracy: 48.64%
End of Epoch 29:
Privacy Budget: Epsilon = 2.2420
Train Loss: 1.4600, Train Accuracy: 48.72%
Test Loss: 1.4697, Test Accuracy: 48.64%
Epoch: 30, Train Loss: 1.4612, Train Accuracy: 49.06%
Epoch: 30, Test Loss: 1.4878, Test Accuracy: 48.44%
Averaging model parameters...
End of Epoch 30:
Privacy Budget: Epsilon = 2.2823
Train Loss: 1.4612, Train Accuracy: 49.06%
Test Loss: 1.4878, Test Accuracy: 48.44%
Epoch: 31, Train Loss: 1.4626, Train Accuracy: 49.10%
Epoch: 31, Test Loss: 1.4844, Test Accuracy: 49.09%


End of Epoch 31:
Privacy Budget: Epsilon = 2.3222
Train Loss: 1.4626, Train Accuracy: 49.10%
Test Loss: 1.4844, Test Accuracy: 49.09%

Parameter averaging+ grouped grad clipp---- grouped_gradient_clipping.py

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...\n")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    # Print statistics for the epoch
    print(f'End of Epoch {epoch}:\n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f}\n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%\n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%\n')

import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()



Epoch: 1, Train Loss: 2.1057, Train Accuracy: 20.05%
Epoch: 1, Test Loss: 1.9697, Test Accuracy: 26.04%
End of Epoch 1:

Privacy Budget: Epsilon = 0.4951

Train Loss: 2.1057, Train Accuracy: 20.05%

Test Loss: 1.9697, Test Accuracy: 26.04%

Epoch: 2, Train Loss: 1.9076, Train Accuracy: 27.74%
Epoch: 2, Test Loss: 1.8368, Test Accuracy: 30.65%
End of Epoch 2:

Privacy Budget: Epsilon = 0.6385

Train Loss: 1.9076, Train Accuracy: 27.74%

Test Loss: 1.8368, Test Accuracy: 30.65%

Epoch: 3, Train Loss: 1.7983, Train Accuracy: 31.05%
Epoch: 3, Test Loss: 1.7697, Test Accuracy: 31.61%
End of Epoch 3:

Privacy Budget: Epsilon = 0.7527

Train Loss: 1.7983, Train Accuracy: 31.05%

Test Loss: 1.7697, Test Accuracy: 31.61%

Epoch: 4, Train Loss: 1.7453, Train Accuracy: 32.75%
Epoch: 4, Test Loss: 1.7251, Test Accuracy: 33.70%
End of Epoch 4:

Privacy Budget: Epsilon = 0.8518

Train Loss: 1.7453, Train Accuracy: 32.75%

Test Loss: 1.7251, Test Accuracy: 33.70%

Epoch: 5, Train Loss: 1.7041, Train Accuracy: 33.91%
Epoch: 5, Test Loss: 1.6973, Test Accuracy: 34.39%
End of Epoch 5:

Privacy Budget: Epsilon = 0.9411

Train Loss: 1.7041, Train Accuracy: 33.91%

Test Loss: 1.6973, Test Accuracy: 34.39%

Epoch: 6, Train Loss: 1.6749, Train Accuracy: 35.61%
Epoch: 6, Test Loss: 1.6673, Test Accuracy: 35.68%
End of Epoch 6:

Privacy Budget: Epsilon = 1.0231

Train Loss: 1.6749, Train Accuracy: 35.61%

Test Loss: 1.6673, Test Accuracy: 35.68%

Epoch: 7, Train Loss: 1.6524, Train Accuracy: 36.98%
Epoch: 7, Test Loss: 1.6389, Test Accuracy: 37.98%
End of Epoch 7:

Privacy Budget: Epsilon = 1.0995

Train Loss: 1.6524, Train Accuracy: 36.98%

Test Loss: 1.6389, Test Accuracy: 37.98%

Epoch: 8, Train Loss: 1.6240, Train Accuracy: 39.02%
Epoch: 8, Test Loss: 1.6263, Test Accuracy: 39.02%
End of Epoch 8:

Privacy Budget: Epsilon = 1.1715

Train Loss: 1.6240, Train Accuracy: 39.02%

Test Loss: 1.6263, Test Accuracy: 39.02%

Epoch: 9, Train Loss: 1.6102, Train Accuracy: 39.97%
Epoch: 9, Test Loss: 1.6171, Test Accuracy: 40.11%
End of Epoch 9:

Privacy Budget: Epsilon = 1.2397

Train Loss: 1.6102, Train Accuracy: 39.97%

Test Loss: 1.6171, Test Accuracy: 40.11%

Epoch: 10, Train Loss: 1.5966, Train Accuracy: 40.83%
Epoch: 10, Test Loss: 1.6105, Test Accuracy: 40.57%
Averaging model parameters...

End of Epoch 10:

Privacy Budget: Epsilon = 1.3049

Train Loss: 1.5966, Train Accuracy: 40.83%

Test Loss: 1.6105, Test Accuracy: 40.57%

Epoch: 11, Train Loss: 1.5838, Train Accuracy: 41.31%
Epoch: 11, Test Loss: 1.5903, Test Accuracy: 41.37%
End of Epoch 11:

Privacy Budget: Epsilon = 1.3673

Train Loss: 1.5838, Train Accuracy: 41.31%

Test Loss: 1.5903, Test Accuracy: 41.37%

Epoch: 12, Train Loss: 1.5741, Train Accuracy: 41.72%
Epoch: 12, Test Loss: 1.5984, Test Accuracy: 41.50%
End of Epoch 12:

Privacy Budget: Epsilon = 1.4274

Train Loss: 1.5741, Train Accuracy: 41.72%

Test Loss: 1.5984, Test Accuracy: 41.50%

Epoch: 13, Train Loss: 1.5721, Train Accuracy: 42.09%
Epoch: 13, Test Loss: 1.5668, Test Accuracy: 43.01%
End of Epoch 13:

Privacy Budget: Epsilon = 1.4854

Train Loss: 1.5721, Train Accuracy: 42.09%

Test Loss: 1.5668, Test Accuracy: 43.01%

Epoch: 14, Train Loss: 1.5589, Train Accuracy: 43.09%
Epoch: 14, Test Loss: 1.5826, Test Accuracy: 41.96%
End of Epoch 14:

Privacy Budget: Epsilon = 1.5415

Train Loss: 1.5589, Train Accuracy: 43.09%

Test Loss: 1.5826, Test Accuracy: 41.96%

Epoch: 15, Train Loss: 1.5504, Train Accuracy: 42.61%
Epoch: 15, Test Loss: 1.5769, Test Accuracy: 42.69%
End of Epoch 15:

Privacy Budget: Epsilon = 1.5959

Train Loss: 1.5504, Train Accuracy: 42.61%

Test Loss: 1.5769, Test Accuracy: 42.69%

Epoch: 16, Train Loss: 1.5464, Train Accuracy: 43.40%
Epoch: 16, Test Loss: 1.5572, Test Accuracy: 43.26%
End of Epoch 16:

Privacy Budget: Epsilon = 1.6489

Train Loss: 1.5464, Train Accuracy: 43.40%

Test Loss: 1.5572, Test Accuracy: 43.26%

Epoch: 17, Train Loss: 1.5518, Train Accuracy: 43.31%
Epoch: 17, Test Loss: 1.5848, Test Accuracy: 42.03%
End of Epoch 17:

Privacy Budget: Epsilon = 1.7004

Train Loss: 1.5518, Train Accuracy: 43.31%

Test Loss: 1.5848, Test Accuracy: 42.03%

Epoch: 18, Train Loss: 1.5270, Train Accuracy: 44.06%
Epoch: 18, Test Loss: 1.5354, Test Accuracy: 44.00%
End of Epoch 18:

Privacy Budget: Epsilon = 1.7507

Train Loss: 1.5270, Train Accuracy: 44.06%

Test Loss: 1.5354, Test Accuracy: 44.00%

Epoch: 19, Train Loss: 1.5329, Train Accuracy: 44.10%
Epoch: 19, Test Loss: 1.5592, Test Accuracy: 43.54%
End of Epoch 19:

Privacy Budget: Epsilon = 1.7998

Train Loss: 1.5329, Train Accuracy: 44.10%

Test Loss: 1.5592, Test Accuracy: 43.54%

Epoch: 20, Train Loss: 1.5248, Train Accuracy: 44.20%
Epoch: 20, Test Loss: 1.5413, Test Accuracy: 44.64%
Averaging model parameters...

End of Epoch 20:

Privacy Budget: Epsilon = 1.8479

Train Loss: 1.5248, Train Accuracy: 44.20%

Test Loss: 1.5413, Test Accuracy: 44.64%

Epoch: 21, Train Loss: 1.5124, Train Accuracy: 44.72%
Epoch: 21, Test Loss: 1.5306, Test Accuracy: 44.66%
End of Epoch 21:

Privacy Budget: Epsilon = 1.8949

Train Loss: 1.5124, Train Accuracy: 44.72%

Test Loss: 1.5306, Test Accuracy: 44.66%

Epoch: 22, Train Loss: 1.5166, Train Accuracy: 45.16%
Epoch: 22, Test Loss: 1.5520, Test Accuracy: 43.68%
End of Epoch 22:

Privacy Budget: Epsilon = 1.9410

Train Loss: 1.5166, Train Accuracy: 45.16%

Test Loss: 1.5520, Test Accuracy: 43.68%

Epoch: 23, Train Loss: 1.5130, Train Accuracy: 44.95%
Epoch: 23, Test Loss: 1.5162, Test Accuracy: 45.63%
End of Epoch 23:

Privacy Budget: Epsilon = 1.9863

Train Loss: 1.5130, Train Accuracy: 44.95%

Test Loss: 1.5162, Test Accuracy: 45.63%

Epoch: 24, Train Loss: 1.5086, Train Accuracy: 45.36%
Epoch: 24, Test Loss: 1.5213, Test Accuracy: 45.35%
End of Epoch 24:

Privacy Budget: Epsilon = 2.0307

Train Loss: 1.5086, Train Accuracy: 45.36%

Test Loss: 1.5213, Test Accuracy: 45.35%

Epoch: 25, Train Loss: 1.4998, Train Accuracy: 45.69%
Epoch: 25, Test Loss: 1.5179, Test Accuracy: 45.69%
End of Epoch 25:

Privacy Budget: Epsilon = 2.0743

Train Loss: 1.4998, Train Accuracy: 45.69%

Test Loss: 1.5179, Test Accuracy: 45.69%

Epoch: 26, Train Loss: 1.5084, Train Accuracy: 45.75%
Epoch: 26, Test Loss: 1.5063, Test Accuracy: 45.81%
End of Epoch 26:

Privacy Budget: Epsilon = 2.1172

Train Loss: 1.5084, Train Accuracy: 45.75%

Test Loss: 1.5063, Test Accuracy: 45.81%

Epoch: 27, Train Loss: 1.5057, Train Accuracy: 45.61%
Epoch: 27, Test Loss: 1.5222, Test Accuracy: 45.56%
End of Epoch 27:

Privacy Budget: Epsilon = 2.1594

Train Loss: 1.5057, Train Accuracy: 45.61%

Test Loss: 1.5222, Test Accuracy: 45.56%

Epoch: 28, Train Loss: 1.5023, Train Accuracy: 46.16%
Epoch: 28, Test Loss: 1.5227, Test Accuracy: 45.65%
End of Epoch 28:

Privacy Budget: Epsilon = 2.2010

Train Loss: 1.5023, Train Accuracy: 46.16%

Test Loss: 1.5227, Test Accuracy: 45.65%

Epoch: 29, Train Loss: 1.5129, Train Accuracy: 45.77%
Epoch: 29, Test Loss: 1.5326, Test Accuracy: 45.57%
End of Epoch 29:

Privacy Budget: Epsilon = 2.2420

Train Loss: 1.5129, Train Accuracy: 45.77%

Test Loss: 1.5326, Test Accuracy: 45.57%

Epoch: 30, Train Loss: 1.5024, Train Accuracy: 46.42%
Epoch: 30, Test Loss: 1.5324, Test Accuracy: 46.20%
Averaging model parameters...

End of Epoch 30:

Privacy Budget: Epsilon = 2.2823

Train Loss: 1.5024, Train Accuracy: 46.42%

Test Loss: 1.5324, Test Accuracy: 46.20%

Epoch: 31, Train Loss: 1.5011, Train Accuracy: 46.52%
Epoch: 31, Test Loss: 1.5103, Test Accuracy: 46.60%
End of Epoch 31:

Privacy Budget: Epsilon = 2.3222

Train Loss: 1.5011, Train Accuracy: 46.52%

Test Loss: 1.5103, Test Accuracy: 46.60%

Epoch: 32, Train Loss: 1.4985, Train Accuracy: 46.79%
Epoch: 32, Test Loss: 1.5118, Test Accuracy: 46.51%
End of Epoch 32:

Privacy Budget: Epsilon = 2.3614

Train Loss: 1.4985, Train Accuracy: 46.79%

Test Loss: 1.5118, Test Accuracy: 46.51%

Epoch: 33, Train Loss: 1.5039, Train Accuracy: 46.77%
Epoch: 33, Test Loss: 1.5345, Test Accuracy: 45.65%
End of Epoch 33:

Privacy Budget: Epsilon = 2.4002

Train Loss: 1.5039, Train Accuracy: 46.77%

Test Loss: 1.5345, Test Accuracy: 45.65%

Epoch: 34, Train Loss: 1.4961, Train Accuracy: 46.84%
Epoch: 34, Test Loss: 1.5206, Test Accuracy: 46.73%
End of Epoch 34:

Privacy Budget: Epsilon = 2.4385

Train Loss: 1.4961, Train Accuracy: 46.84%

Test Loss: 1.5206, Test Accuracy: 46.73%

Epoch: 35, Train Loss: 1.4949, Train Accuracy: 47.12%
Epoch: 35, Test Loss: 1.5377, Test Accuracy: 45.98%
End of Epoch 35:

Privacy Budget: Epsilon = 2.4763

Train Loss: 1.4949, Train Accuracy: 47.12%

Test Loss: 1.5377, Test Accuracy: 45.98%

Epoch: 36, Train Loss: 1.4840, Train Accuracy: 47.02%
Epoch: 36, Test Loss: 1.5434, Test Accuracy: 45.75%
End of Epoch 36:

Privacy Budget: Epsilon = 2.5137

Train Loss: 1.4840, Train Accuracy: 47.02%

Test Loss: 1.5434, Test Accuracy: 45.75%

Epoch: 37, Train Loss: 1.4900, Train Accuracy: 47.14%
Epoch: 37, Test Loss: 1.5276, Test Accuracy: 46.88%
End of Epoch 37:

Privacy Budget: Epsilon = 2.5507

Train Loss: 1.4900, Train Accuracy: 47.14%

Test Loss: 1.5276, Test Accuracy: 46.88%

Epoch: 38, Train Loss: 1.4937, Train Accuracy: 47.13%
Epoch: 38, Test Loss: 1.5133, Test Accuracy: 47.15%
End of Epoch 38:

Privacy Budget: Epsilon = 2.5872

Train Loss: 1.4937, Train Accuracy: 47.13%

Test Loss: 1.5133, Test Accuracy: 47.15%

Epoch: 39, Train Loss: 1.4936, Train Accuracy: 47.28%
Epoch: 39, Test Loss: 1.5017, Test Accuracy: 46.82%
End of Epoch 39:

Privacy Budget: Epsilon = 2.6233

Train Loss: 1.4936, Train Accuracy: 47.28%

Test Loss: 1.5017, Test Accuracy: 46.82%

Epoch: 40, Train Loss: 1.4939, Train Accuracy: 47.55%
Epoch: 40, Test Loss: 1.5395, Test Accuracy: 46.01%
Averaging model parameters...

End of Epoch 40:

Privacy Budget: Epsilon = 2.6591

Train Loss: 1.4939, Train Accuracy: 47.55%

Test Loss: 1.5395, Test Accuracy: 46.01%

Epoch: 41, Train Loss: 1.4973, Train Accuracy: 47.50%
Epoch: 41, Test Loss: 1.5036, Test Accuracy: 47.48%
End of Epoch 41:

Privacy Budget: Epsilon = 2.6945

Train Loss: 1.4973, Train Accuracy: 47.50%

Test Loss: 1.5036, Test Accuracy: 47.48%

Epoch: 42, Train Loss: 1.4920, Train Accuracy: 47.76%
Epoch: 42, Test Loss: 1.5241, Test Accuracy: 47.16%
End of Epoch 42:

Privacy Budget: Epsilon = 2.7295

Train Loss: 1.4920, Train Accuracy: 47.76%

Test Loss: 1.5241, Test Accuracy: 47.16%

Epoch: 43, Train Loss: 1.4929, Train Accuracy: 47.73%
Epoch: 43, Test Loss: 1.5543, Test Accuracy: 47.28%
End of Epoch 43:

Privacy Budget: Epsilon = 2.7642

Train Loss: 1.4929, Train Accuracy: 47.73%

Test Loss: 1.5543, Test Accuracy: 47.28%

Epoch: 44, Train Loss: 1.4950, Train Accuracy: 47.66%
Epoch: 44, Test Loss: 1.5229, Test Accuracy: 46.97%
End of Epoch 44:

Privacy Budget: Epsilon = 2.7986

Train Loss: 1.4950, Train Accuracy: 47.66%

Test Loss: 1.5229, Test Accuracy: 46.97%

Epoch: 45, Train Loss: 1.4940, Train Accuracy: 47.84%
Epoch: 45, Test Loss: 1.5100, Test Accuracy: 47.15%
End of Epoch 45:

Privacy Budget: Epsilon = 2.8327

Train Loss: 1.4940, Train Accuracy: 47.84%

Test Loss: 1.5100, Test Accuracy: 47.15%

Epoch: 46, Train Loss: 1.5048, Train Accuracy: 47.58%
Epoch: 46, Test Loss: 1.5387, Test Accuracy: 46.64%
End of Epoch 46:

Privacy Budget: Epsilon = 2.8664

Train Loss: 1.5048, Train Accuracy: 47.58%

Test Loss: 1.5387, Test Accuracy: 46.64%

Epoch: 47, Train Loss: 1.4996, Train Accuracy: 48.03%
Epoch: 47, Test Loss: 1.5237, Test Accuracy: 47.14%
End of Epoch 47:

Privacy Budget: Epsilon = 2.8998

Train Loss: 1.4996, Train Accuracy: 48.03%

Test Loss: 1.5237, Test Accuracy: 47.14%

Epoch: 48, Train Loss: 1.4999, Train Accuracy: 47.80%
Epoch: 48, Test Loss: 1.5191, Test Accuracy: 47.54%
End of Epoch 48:

Privacy Budget: Epsilon = 2.9330

Train Loss: 1.4999, Train Accuracy: 47.80%

Test Loss: 1.5191, Test Accuracy: 47.54%

Epoch: 49, Train Loss: 1.5010, Train Accuracy: 48.23%
Epoch: 49, Test Loss: 1.5093, Test Accuracy: 47.89%
End of Epoch 49:

Privacy Budget: Epsilon = 2.9659

Train Loss: 1.5010, Train Accuracy: 48.23%

Test Loss: 1.5093, Test Accuracy: 47.89%

Epoch: 1, Train Loss: 2.1057, Train Accuracy: 20.05%
Epoch: 1, Test Loss: 1.9697, Test Accuracy: 26.04%
End of Epoch 1:

Privacy Budget: Epsilon = 0.4951

Train Loss: 2.1057, Train Accuracy: 20.05%

Test Loss: 1.9697, Test Accuracy: 26.04%

Epoch: 2, Train Loss: 1.9076, Train Accuracy: 27.74%
Epoch: 2, Test Loss: 1.8368, Test Accuracy: 30.65%
End of Epoch 2:

Privacy Budget: Epsilon = 0.6385

Train Loss: 1.9076, Train Accuracy: 27.74%

Test Loss: 1.8368, Test Accuracy: 30.65%

Epoch: 3, Train Loss: 1.7983, Train Accuracy: 31.05%
Epoch: 3, Test Loss: 1.7697, Test Accuracy: 31.61%
End of Epoch 3:

Privacy Budget: Epsilon = 0.7527

Train Loss: 1.7983, Train Accuracy: 31.05%

Test Loss: 1.7697, Test Accuracy: 31.61%

Epoch: 4, Train Loss: 1.7453, Train Accuracy: 32.75%
Epoch: 4, Test Loss: 1.7251, Test Accuracy: 33.70%
End of Epoch 4:

Privacy Budget: Epsilon = 0.8518

Train Loss: 1.7453, Train Accuracy: 32.75%

Test Loss: 1.7251, Test Accuracy: 33.70%

Epoch: 5, Train Loss: 1.7041, Train Accuracy: 33.91%
Epoch: 5, Test Loss: 1.6973, Test Accuracy: 34.39%
End of Epoch 5:

Privacy Budget: Epsilon = 0.9411

Train Loss: 1.7041, Train Accuracy: 33.91%

Test Loss: 1.6973, Test Accuracy: 34.39%

Epoch: 6, Train Loss: 1.6749, Train Accuracy: 35.61%
Epoch: 6, Test Loss: 1.6673, Test Accuracy: 35.68%
End of Epoch 6:

Privacy Budget: Epsilon = 1.0231

Train Loss: 1.6749, Train Accuracy: 35.61%

Test Loss: 1.6673, Test Accuracy: 35.68%

Epoch: 7, Train Loss: 1.6524, Train Accuracy: 36.98%
Epoch: 7, Test Loss: 1.6389, Test Accuracy: 37.98%
End of Epoch 7:

Privacy Budget: Epsilon = 1.0995

Train Loss: 1.6524, Train Accuracy: 36.98%

Test Loss: 1.6389, Test Accuracy: 37.98%

Epoch: 8, Train Loss: 1.6240, Train Accuracy: 39.02%
Epoch: 8, Test Loss: 1.6263, Test Accuracy: 39.02%
End of Epoch 8:

Privacy Budget: Epsilon = 1.1715

Train Loss: 1.6240, Train Accuracy: 39.02%

Test Loss: 1.6263, Test Accuracy: 39.02%

Epoch: 9, Train Loss: 1.6102, Train Accuracy: 39.97%
Epoch: 9, Test Loss: 1.6171, Test Accuracy: 40.11%
End of Epoch 9:

Privacy Budget: Epsilon = 1.2397

Train Loss: 1.6102, Train Accuracy: 39.97%

Test Loss: 1.6171, Test Accuracy: 40.11%

Epoch: 10, Train Loss: 1.5966, Train Accuracy: 40.83%
Epoch: 10, Test Loss: 1.6105, Test Accuracy: 40.57%
Averaging model parameters...

End of Epoch 10:

Privacy Budget: Epsilon = 1.3049

Train Loss: 1.5966, Train Accuracy: 40.83%

Test Loss: 1.6105, Test Accuracy: 40.57%

Epoch: 11, Train Loss: 1.5838, Train Accuracy: 41.31%
Epoch: 11, Test Loss: 1.5903, Test Accuracy: 41.37%
End of Epoch 11:

Privacy Budget: Epsilon = 1.3673

Train Loss: 1.5838, Train Accuracy: 41.31%

Test Loss: 1.5903, Test Accuracy: 41.37%

Epoch: 12, Train Loss: 1.5741, Train Accuracy: 41.72%
Epoch: 12, Test Loss: 1.5984, Test Accuracy: 41.50%
End of Epoch 12:

Privacy Budget: Epsilon = 1.4274

Train Loss: 1.5741, Train Accuracy: 41.72%

Test Loss: 1.5984, Test Accuracy: 41.50%

Epoch: 13, Train Loss: 1.5721, Train Accuracy: 42.09%
Epoch: 13, Test Loss: 1.5668, Test Accuracy: 43.01%
End of Epoch 13:

Privacy Budget: Epsilon = 1.4854

Train Loss: 1.5721, Train Accuracy: 42.09%

Test Loss: 1.5668, Test Accuracy: 43.01%

Epoch: 14, Train Loss: 1.5589, Train Accuracy: 43.09%
Epoch: 14, Test Loss: 1.5826, Test Accuracy: 41.96%
End of Epoch 14:

Privacy Budget: Epsilon = 1.5415

Train Loss: 1.5589, Train Accuracy: 43.09%

Test Loss: 1.5826, Test Accuracy: 41.96%

Epoch: 15, Train Loss: 1.5504, Train Accuracy: 42.61%
Epoch: 15, Test Loss: 1.5769, Test Accuracy: 42.69%
End of Epoch 15:

Privacy Budget: Epsilon = 1.5959

Train Loss: 1.5504, Train Accuracy: 42.61%

Test Loss: 1.5769, Test Accuracy: 42.69%

Epoch: 16, Train Loss: 1.5464, Train Accuracy: 43.40%
Epoch: 16, Test Loss: 1.5572, Test Accuracy: 43.26%
End of Epoch 16:

Privacy Budget: Epsilon = 1.6489

Train Loss: 1.5464, Train Accuracy: 43.40%

Test Loss: 1.5572, Test Accuracy: 43.26%

Epoch: 17, Train Loss: 1.5518, Train Accuracy: 43.31%
Epoch: 17, Test Loss: 1.5848, Test Accuracy: 42.03%
End of Epoch 17:

Privacy Budget: Epsilon = 1.7004

Train Loss: 1.5518, Train Accuracy: 43.31%

Test Loss: 1.5848, Test Accuracy: 42.03%

Epoch: 18, Train Loss: 1.5270, Train Accuracy: 44.06%
Epoch: 18, Test Loss: 1.5354, Test Accuracy: 44.00%
End of Epoch 18:

Privacy Budget: Epsilon = 1.7507

Train Loss: 1.5270, Train Accuracy: 44.06%

Test Loss: 1.5354, Test Accuracy: 44.00%

Epoch: 19, Train Loss: 1.5329, Train Accuracy: 44.10%
Epoch: 19, Test Loss: 1.5592, Test Accuracy: 43.54%
End of Epoch 19:

Privacy Budget: Epsilon = 1.7998

Train Loss: 1.5329, Train Accuracy: 44.10%

Test Loss: 1.5592, Test Accuracy: 43.54%

Epoch: 20, Train Loss: 1.5248, Train Accuracy: 44.20%
Epoch: 20, Test Loss: 1.5413, Test Accuracy: 44.64%
Averaging model parameters...

End of Epoch 20:

Privacy Budget: Epsilon = 1.8479

Train Loss: 1.5248, Train Accuracy: 44.20%

Test Loss: 1.5413, Test Accuracy: 44.64%

Epoch: 21, Train Loss: 1.5124, Train Accuracy: 44.72%
Epoch: 21, Test Loss: 1.5306, Test Accuracy: 44.66%
End of Epoch 21:

Privacy Budget: Epsilon = 1.8949

Train Loss: 1.5124, Train Accuracy: 44.72%

Test Loss: 1.5306, Test Accuracy: 44.66%

Epoch: 22, Train Loss: 1.5166, Train Accuracy: 45.16%
Epoch: 22, Test Loss: 1.5520, Test Accuracy: 43.68%
End of Epoch 22:

Privacy Budget: Epsilon = 1.9410

Train Loss: 1.5166, Train Accuracy: 45.16%

Test Loss: 1.5520, Test Accuracy: 43.68%

Epoch: 23, Train Loss: 1.5130, Train Accuracy: 44.95%
Epoch: 23, Test Loss: 1.5162, Test Accuracy: 45.63%
End of Epoch 23:

Privacy Budget: Epsilon = 1.9863

Train Loss: 1.5130, Train Accuracy: 44.95%

Test Loss: 1.5162, Test Accuracy: 45.63%

Epoch: 24, Train Loss: 1.5086, Train Accuracy: 45.36%
Epoch: 24, Test Loss: 1.5213, Test Accuracy: 45.35%
End of Epoch 24:

Privacy Budget: Epsilon = 2.0307

Train Loss: 1.5086, Train Accuracy: 45.36%

Test Loss: 1.5213, Test Accuracy: 45.35%

Epoch: 25, Train Loss: 1.4998, Train Accuracy: 45.69%
Epoch: 25, Test Loss: 1.5179, Test Accuracy: 45.69%
End of Epoch 25:

Privacy Budget: Epsilon = 2.0743

Train Loss: 1.4998, Train Accuracy: 45.69%

Test Loss: 1.5179, Test Accuracy: 45.69%

Epoch: 26, Train Loss: 1.5084, Train Accuracy: 45.75%
Epoch: 26, Test Loss: 1.5063, Test Accuracy: 45.81%
End of Epoch 26:

Privacy Budget: Epsilon = 2.1172

Train Loss: 1.5084, Train Accuracy: 45.75%

Test Loss: 1.5063, Test Accuracy: 45.81%

Epoch: 27, Train Loss: 1.5057, Train Accuracy: 45.61%
Epoch: 27, Test Loss: 1.5222, Test Accuracy: 45.56%
End of Epoch 27:

Privacy Budget: Epsilon = 2.1594

Train Loss: 1.5057, Train Accuracy: 45.61%

Test Loss: 1.5222, Test Accuracy: 45.56%

Epoch: 28, Train Loss: 1.5023, Train Accuracy: 46.16%
Epoch: 28, Test Loss: 1.5227, Test Accuracy: 45.65%
End of Epoch 28:

Privacy Budget: Epsilon = 2.2010

Train Loss: 1.5023, Train Accuracy: 46.16%

Test Loss: 1.5227, Test Accuracy: 45.65%

Epoch: 29, Train Loss: 1.5129, Train Accuracy: 45.77%
Epoch: 29, Test Loss: 1.5326, Test Accuracy: 45.57%
End of Epoch 29:

Privacy Budget: Epsilon = 2.2420

Train Loss: 1.5129, Train Accuracy: 45.77%

Test Loss: 1.5326, Test Accuracy: 45.57%

Epoch: 30, Train Loss: 1.5024, Train Accuracy: 46.42%
Epoch: 30, Test Loss: 1.5324, Test Accuracy: 46.20%
Averaging model parameters...

End of Epoch 30:

Privacy Budget: Epsilon = 2.2823

Train Loss: 1.5024, Train Accuracy: 46.42%

Test Loss: 1.5324, Test Accuracy: 46.20%

Epoch: 31, Train Loss: 1.5011, Train Accuracy: 46.52%
Epoch: 31, Test Loss: 1.5103, Test Accuracy: 46.60%
End of Epoch 31:

Privacy Budget: Epsilon = 2.3222

Train Loss: 1.5011, Train Accuracy: 46.52%

Test Loss: 1.5103, Test Accuracy: 46.60%

Epoch: 32, Train Loss: 1.4985, Train Accuracy: 46.79%
Epoch: 32, Test Loss: 1.5118, Test Accuracy: 46.51%
End of Epoch 32:

Privacy Budget: Epsilon = 2.3614

Train Loss: 1.4985, Train Accuracy: 46.79%

Test Loss: 1.5118, Test Accuracy: 46.51%

Epoch: 33, Train Loss: 1.5039, Train Accuracy: 46.77%
Epoch: 33, Test Loss: 1.5345, Test Accuracy: 45.65%
End of Epoch 33:

Privacy Budget: Epsilon = 2.4002

Train Loss: 1.5039, Train Accuracy: 46.77%

Test Loss: 1.5345, Test Accuracy: 45.65%

Epoch: 34, Train Loss: 1.4961, Train Accuracy: 46.84%
Epoch: 34, Test Loss: 1.5206, Test Accuracy: 46.73%
End of Epoch 34:

Privacy Budget: Epsilon = 2.4385

Train Loss: 1.4961, Train Accuracy: 46.84%

Test Loss: 1.5206, Test Accuracy: 46.73%

Epoch: 35, Train Loss: 1.4949, Train Accuracy: 47.12%
Epoch: 35, Test Loss: 1.5377, Test Accuracy: 45.98%
End of Epoch 35:

Privacy Budget: Epsilon = 2.4763

Train Loss: 1.4949, Train Accuracy: 47.12%

Test Loss: 1.5377, Test Accuracy: 45.98%

Epoch: 36, Train Loss: 1.4840, Train Accuracy: 47.02%
Epoch: 36, Test Loss: 1.5434, Test Accuracy: 45.75%
End of Epoch 36:

Privacy Budget: Epsilon = 2.5137

Train Loss: 1.4840, Train Accuracy: 47.02%

Test Loss: 1.5434, Test Accuracy: 45.75%

Epoch: 37, Train Loss: 1.4900, Train Accuracy: 47.14%
Epoch: 37, Test Loss: 1.5276, Test Accuracy: 46.88%
End of Epoch 37:

Privacy Budget: Epsilon = 2.5507

Train Loss: 1.4900, Train Accuracy: 47.14%

Test Loss: 1.5276, Test Accuracy: 46.88%

Epoch: 38, Train Loss: 1.4937, Train Accuracy: 47.13%
Epoch: 38, Test Loss: 1.5133, Test Accuracy: 47.15%
End of Epoch 38:

Privacy Budget: Epsilon = 2.5872

Train Loss: 1.4937, Train Accuracy: 47.13%

Test Loss: 1.5133, Test Accuracy: 47.15%

Epoch: 39, Train Loss: 1.4936, Train Accuracy: 47.28%
Epoch: 39, Test Loss: 1.5017, Test Accuracy: 46.82%
End of Epoch 39:

Privacy Budget: Epsilon = 2.6233

Train Loss: 1.4936, Train Accuracy: 47.28%

Test Loss: 1.5017, Test Accuracy: 46.82%

Epoch: 40, Train Loss: 1.4939, Train Accuracy: 47.55%
Epoch: 40, Test Loss: 1.5395, Test Accuracy: 46.01%
Averaging model parameters...

End of Epoch 40:

Privacy Budget: Epsilon = 2.6591

Train Loss: 1.4939, Train Accuracy: 47.55%

Test Loss: 1.5395, Test Accuracy: 46.01%

Epoch: 41, Train Loss: 1.4973, Train Accuracy: 47.50%
Epoch: 41, Test Loss: 1.5036, Test Accuracy: 47.48%
End of Epoch 41:

Privacy Budget: Epsilon = 2.6945

Train Loss: 1.4973, Train Accuracy: 47.50%

Test Loss: 1.5036, Test Accuracy: 47.48%

Epoch: 42, Train Loss: 1.4920, Train Accuracy: 47.76%
Epoch: 42, Test Loss: 1.5241, Test Accuracy: 47.16%
End of Epoch 42:

Privacy Budget: Epsilon = 2.7295

Train Loss: 1.4920, Train Accuracy: 47.76%

Test Loss: 1.5241, Test Accuracy: 47.16%

Epoch: 43, Train Loss: 1.4929, Train Accuracy: 47.73%
Epoch: 43, Test Loss: 1.5543, Test Accuracy: 47.28%
End of Epoch 43:

Privacy Budget: Epsilon = 2.7642

Train Loss: 1.4929, Train Accuracy: 47.73%

Test Loss: 1.5543, Test Accuracy: 47.28%

Epoch: 44, Train Loss: 1.4950, Train Accuracy: 47.66%
Epoch: 44, Test Loss: 1.5229, Test Accuracy: 46.97%
End of Epoch 44:

Privacy Budget: Epsilon = 2.7986

Train Loss: 1.4950, Train Accuracy: 47.66%

Test Loss: 1.5229, Test Accuracy: 46.97%

Epoch: 45, Train Loss: 1.4940, Train Accuracy: 47.84%
Epoch: 45, Test Loss: 1.5100, Test Accuracy: 47.15%
End of Epoch 45:

Privacy Budget: Epsilon = 2.8327

Train Loss: 1.4940, Train Accuracy: 47.84%

Test Loss: 1.5100, Test Accuracy: 47.15%

Epoch: 46, Train Loss: 1.5048, Train Accuracy: 47.58%
Epoch: 46, Test Loss: 1.5387, Test Accuracy: 46.64%
End of Epoch 46:

Privacy Budget: Epsilon = 2.8664

Train Loss: 1.5048, Train Accuracy: 47.58%

Test Loss: 1.5387, Test Accuracy: 46.64%

Epoch: 47, Train Loss: 1.4996, Train Accuracy: 48.03%
Epoch: 47, Test Loss: 1.5237, Test Accuracy: 47.14%
End of Epoch 47:

Privacy Budget: Epsilon = 2.8998

Train Loss: 1.4996, Train Accuracy: 48.03%

Test Loss: 1.5237, Test Accuracy: 47.14%

Epoch: 48, Train Loss: 1.4999, Train Accuracy: 47.80%
Epoch: 48, Test Loss: 1.5191, Test Accuracy: 47.54%
End of Epoch 48:

Privacy Budget: Epsilon = 2.9330

Train Loss: 1.4999, Train Accuracy: 47.80%

Test Loss: 1.5191, Test Accuracy: 47.54%

Epoch: 49, Train Loss: 1.5010, Train Accuracy: 48.23%
Epoch: 49, Test Loss: 1.5093, Test Accuracy: 47.89%
End of Epoch 49:

Privacy Budget: Epsilon = 2.9659

Train Loss: 1.5010, Train Accuracy: 48.23%

Test Loss: 1.5093, Test Accuracy: 47.89%

[sg27405@csci-cscuda final]$


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extracting the data from the given logs
epochs = np.arange(1, 51)
train_accuracy_vals = [
    20.05, 27.74, 31.05, 32.75, 33.91, 35.61, 36.98, 39.02, 39.97, 40.83,
    41.31, 41.72, 42.09, 43.09, 42.61, 43.40, 43.31, 44.06, 44.10, 44.20,
    44.72, 45.16, 44.95, 45.36, 45.69, 45.75, 45.61, 46.16, 45.77, 46.42,
    46.52, 46.79, 46.77, 46.84, 47.12, 47.02, 47.14, 47.13, 47.28, 47.55,
    47.50, 47.76, 47.73, 47.66, 47.84, 47.58, 48.03, 47.80, 48.23
]
test_accuracy_vals = [
    26.04, 30.65, 31.61, 33.70, 34.39, 35.68, 37.98, 39.02, 40.11, 40.57,
    41.37, 41.50, 43.01, 41.96, 42.69, 43.26, 42.03, 44.00, 43.54, 44.64,
    44.66, 43.68, 45.63, 45.35, 45.69, 45.81, 45.56, 45.65, 45.57, 46.20,
    46.60, 46.51, 45.65, 46.73, 45.98, 45.75, 46.88, 47.15, 46.82, 46.01,
    47.48, 47.16, 47.28, 46.97, 47.15, 46.64, 47.14, 47.54, 47.89
]

# Extracting privacy budget epsilon values
epsilon_vals = [
    0.4951, 0.6385, 0.7527, 0.8518, 0.9411, 1.0231, 1.0995, 1.1715, 1.2397, 1.3049,
    1.3673, 1.4274, 1.4854, 1.5415, 1.5959, 1.6489, 1.7004, 1.7507, 1.7998, 1.8479,
    1.8949, 1.9410, 1.9863, 2.0307, 2.0743, 2.1172, 2.1594, 2.2010, 2.2420, 2.2823,
    2.3222, 2.3614, 2.4002, 2.4385, 2.4763, 2.5137, 2.5507, 2.5872, 2.6233, 2.6591,
    2.6945, 2.7295, 2.7642, 2.7986, 2.8327, 2.8664, 2.8998, 2.9330, 2.9659
]

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.xticks(epochs)  # Set x-ticks to show all epochs
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='o')
plt.xlabel('Privacy Budget (Epsilon)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.legend()
plt.grid(True)
plt.show()


LR scheduler added

 T_max:
Definition: The number of epochs required for the learning rate to complete one full cosine cycle.
Effect: Determines how gradually the learning rate decays.
Usage in your case:
Since your training loop runs for 50 epochs, setting T_max=50 ensures the learning rate decreases smoothly to the minimum (eta_min) by the end of training.
2. eta_min:
Definition: The minimum learning rate at the end of the cosine decay.
Effect: Acts as a floor for the learning rate, preventing it from becoming too small (which can stall training).
Usage in your case:
eta_min=0.0001 ensures the learning rate does not drop below 0.0001, even at the last epoch.


LR=0.01. BATCH SIZE=256



In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]


def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)


# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')

import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


Epoch: 1, Train Loss: 2.1210, Train Accuracy: 20.06%

Epoch: 1, Test Loss: 1.9898, Test Accuracy: 26.24%

End of Epoch 1:

Privacy Budget: Epsilon = 0.4951

Train Loss: 2.1210, Train Accuracy: 20.06%

Test Loss: 1.9898, Test Accuracy: 26.24%

Learning Rate: 0.009990

Epoch: 2, Train Loss: 1.9328, Train Accuracy: 27.77%

Epoch: 2, Test Loss: 1.8692, Test Accuracy: 30.06%

End of Epoch 2:

Privacy Budget: Epsilon = 0.6385

Train Loss: 1.9328, Train Accuracy: 27.77%

Test Loss: 1.8692, Test Accuracy: 30.06%

Learning Rate: 0.009961

Epoch: 3, Train Loss: 1.8274, Train Accuracy: 30.89%

Epoch: 3, Test Loss: 1.7854, Test Accuracy: 31.52%

End of Epoch 3:

Privacy Budget: Epsilon = 0.7527

Train Loss: 1.8274, Train Accuracy: 30.89%

Test Loss: 1.7854, Test Accuracy: 31.52%

Learning Rate: 0.009912

Epoch: 4, Train Loss: 1.7694, Train Accuracy: 32.05%

Epoch: 4, Test Loss: 1.7510, Test Accuracy: 33.21%

End of Epoch 4:

Privacy Budget: Epsilon = 0.8518

Train Loss: 1.7694, Train Accuracy: 32.05%

Test Loss: 1.7510, Test Accuracy: 33.21%

Learning Rate: 0.009844

Epoch: 5, Train Loss: 1.7416, Train Accuracy: 33.78%

Epoch: 5, Test Loss: 1.7326, Test Accuracy: 34.48%

End of Epoch 5:

Privacy Budget: Epsilon = 0.9411

Train Loss: 1.7416, Train Accuracy: 33.78%

Test Loss: 1.7326, Test Accuracy: 34.48%

Learning Rate: 0.009758

Epoch: 6, Train Loss: 1.7219, Train Accuracy: 35.44%

Epoch: 6, Test Loss: 1.7259, Test Accuracy: 34.98%

End of Epoch 6:

Privacy Budget: Epsilon = 1.0231

Train Loss: 1.7219, Train Accuracy: 35.44%

Test Loss: 1.7259, Test Accuracy: 34.98%

Learning Rate: 0.009652

Epoch: 7, Train Loss: 1.7199, Train Accuracy: 36.10%

Epoch: 7, Test Loss: 1.7071, Test Accuracy: 36.67%

End of Epoch 7:

Privacy Budget: Epsilon = 1.0995

Train Loss: 1.7199, Train Accuracy: 36.10%

Test Loss: 1.7071, Test Accuracy: 36.67%

Learning Rate: 0.009529

Epoch: 8, Train Loss: 1.6872, Train Accuracy: 36.87%

Epoch: 8, Test Loss: 1.6810, Test Accuracy: 37.11%

End of Epoch 8:

Privacy Budget: Epsilon = 1.1715

Train Loss: 1.6872, Train Accuracy: 36.87%

Test Loss: 1.6810, Test Accuracy: 37.11%

Learning Rate: 0.009388

Epoch: 9, Train Loss: 1.6766, Train Accuracy: 37.21%

Epoch: 9, Test Loss: 1.6630, Test Accuracy: 38.22%

End of Epoch 9:

Privacy Budget: Epsilon = 1.2397

Train Loss: 1.6766, Train Accuracy: 37.21%

Test Loss: 1.6630, Test Accuracy: 38.22%

Learning Rate: 0.009229

Epoch: 10, Train Loss: 1.6547, Train Accuracy: 38.17%

Epoch: 10, Test Loss: 1.6745, Test Accuracy: 37.73%

Averaging model parameters...
End of Epoch 10:

Privacy Budget: Epsilon = 1.3049

Train Loss: 1.6547, Train Accuracy: 38.17%

Test Loss: 1.6745, Test Accuracy: 37.73%

Learning Rate: 0.009055

Epoch: 11, Train Loss: 1.6288, Train Accuracy: 38.90%

Epoch: 11, Test Loss: 1.6262, Test Accuracy: 38.95%

End of Epoch 11:

Privacy Budget: Epsilon = 1.3673

Train Loss: 1.6288, Train Accuracy: 38.90%

Test Loss: 1.6262, Test Accuracy: 38.95%

Learning Rate: 0.008864

Epoch: 12, Train Loss: 1.6225, Train Accuracy: 39.96%

Epoch: 12, Test Loss: 1.6122, Test Accuracy: 40.24%

End of Epoch 12:

Privacy Budget: Epsilon = 1.4274

Train Loss: 1.6225, Train Accuracy: 39.96%

Test Loss: 1.6122, Test Accuracy: 40.24%

Learning Rate: 0.008658

Epoch: 13, Train Loss: 1.6043, Train Accuracy: 40.54%

Epoch: 13, Test Loss: 1.6022, Test Accuracy: 40.79%

End of Epoch 13:

Privacy Budget: Epsilon = 1.4854

Train Loss: 1.6043, Train Accuracy: 40.54%

Test Loss: 1.6022, Test Accuracy: 40.79%

Learning Rate: 0.008439

Epoch: 14, Train Loss: 1.5921, Train Accuracy: 41.07%

Epoch: 14, Test Loss: 1.5942, Test Accuracy: 40.78%

End of Epoch 14:

Privacy Budget: Epsilon = 1.5415

Train Loss: 1.5921, Train Accuracy: 41.07%

Test Loss: 1.5942, Test Accuracy: 40.78%

Learning Rate: 0.008205

Epoch: 15, Train Loss: 1.5885, Train Accuracy: 41.49%

Epoch: 15, Test Loss: 1.5778, Test Accuracy: 41.84%

End of Epoch 15:

Privacy Budget: Epsilon = 1.5959

Train Loss: 1.5885, Train Accuracy: 41.49%

Test Loss: 1.5778, Test Accuracy: 41.84%

Learning Rate: 0.007960

Epoch: 16, Train Loss: 1.5708, Train Accuracy: 42.55%

Epoch: 16, Test Loss: 1.5629, Test Accuracy: 43.02%

End of Epoch 16:

Privacy Budget: Epsilon = 1.6489

Train Loss: 1.5708, Train Accuracy: 42.55%

Test Loss: 1.5629, Test Accuracy: 43.02%

Learning Rate: 0.007702

Epoch: 17, Train Loss: 1.5494, Train Accuracy: 43.43%

Epoch: 17, Test Loss: 1.5537, Test Accuracy: 43.44%

End of Epoch 17:

Privacy Budget: Epsilon = 1.7004

Train Loss: 1.5494, Train Accuracy: 43.43%

Test Loss: 1.5537, Test Accuracy: 43.44%

Learning Rate: 0.007435

Epoch: 18, Train Loss: 1.5358, Train Accuracy: 44.05%

Epoch: 18, Test Loss: 1.5505, Test Accuracy: 43.52%

End of Epoch 18:

Privacy Budget: Epsilon = 1.7507

Train Loss: 1.5358, Train Accuracy: 44.05%

Test Loss: 1.5505, Test Accuracy: 43.52%

Learning Rate: 0.007158

Epoch: 19, Train Loss: 1.5367, Train Accuracy: 44.06%

Epoch: 19, Test Loss: 1.5719, Test Accuracy: 43.02%

End of Epoch 19:

Privacy Budget: Epsilon = 1.7998

Train Loss: 1.5367, Train Accuracy: 44.06%

Test Loss: 1.5719, Test Accuracy: 43.02%

Learning Rate: 0.006872

Epoch: 20, Train Loss: 1.5293, Train Accuracy: 44.66%

Epoch: 20, Test Loss: 1.5375, Test Accuracy: 44.53%

Averaging model parameters...
End of Epoch 20:

Privacy Budget: Epsilon = 1.8479

Train Loss: 1.5293, Train Accuracy: 44.66%

Test Loss: 1.5375, Test Accuracy: 44.53%

Learning Rate: 0.006580

Epoch: 21, Train Loss: 1.5242, Train Accuracy: 44.64%

Epoch: 21, Test Loss: 1.5271, Test Accuracy: 44.78%

End of Epoch 21:

Privacy Budget: Epsilon = 1.8949

Train Loss: 1.5242, Train Accuracy: 44.64%

Test Loss: 1.5271, Test Accuracy: 44.78%

Learning Rate: 0.006281

Epoch: 22, Train Loss: 1.5089, Train Accuracy: 45.12%

Epoch: 22, Test Loss: 1.5211, Test Accuracy: 45.24%

End of Epoch 22:

Privacy Budget: Epsilon = 1.9410

Train Loss: 1.5089, Train Accuracy: 45.12%

Test Loss: 1.5211, Test Accuracy: 45.24%

Learning Rate: 0.005978

Epoch: 23, Train Loss: 1.5079, Train Accuracy: 45.96%

Epoch: 23, Test Loss: 1.5169, Test Accuracy: 45.54%

End of Epoch 23:

Privacy Budget: Epsilon = 1.9863

Train Loss: 1.5079, Train Accuracy: 45.96%

Test Loss: 1.5169, Test Accuracy: 45.54%

Learning Rate: 0.005670

Epoch: 24, Train Loss: 1.5030, Train Accuracy: 46.17%

Epoch: 24, Test Loss: 1.5087, Test Accuracy: 45.65%

End of Epoch 24:

Privacy Budget: Epsilon = 2.0307

Train Loss: 1.5030, Train Accuracy: 46.17%

Test Loss: 1.5087, Test Accuracy: 45.65%

Learning Rate: 0.005361

Epoch: 25, Train Loss: 1.5038, Train Accuracy: 46.26%

Epoch: 25, Test Loss: 1.5287, Test Accuracy: 45.80%

End of Epoch 25:

Privacy Budget: Epsilon = 2.0743

Train Loss: 1.5038, Train Accuracy: 46.26%

Test Loss: 1.5287, Test Accuracy: 45.80%

Learning Rate: 0.005050

Epoch: 26, Train Loss: 1.4834, Train Accuracy: 47.00%

Epoch: 26, Test Loss: 1.5073, Test Accuracy: 46.55%

End of Epoch 26:

Privacy Budget: Epsilon = 2.1172

Train Loss: 1.4834, Train Accuracy: 47.00%

Test Loss: 1.5073, Test Accuracy: 46.55%

Learning Rate: 0.004739

Epoch: 27, Train Loss: 1.4799, Train Accuracy: 46.94%

Epoch: 27, Test Loss: 1.4946, Test Accuracy: 45.96%

End of Epoch 27:

Privacy Budget: Epsilon = 2.1594

Train Loss: 1.4799, Train Accuracy: 46.94%

Test Loss: 1.4946, Test Accuracy: 45.96%

Learning Rate: 0.004430

Epoch: 28, Train Loss: 1.4780, Train Accuracy: 46.95%

Epoch: 28, Test Loss: 1.5009, Test Accuracy: 46.23%

End of Epoch 28:

Privacy Budget: Epsilon = 2.2010

Train Loss: 1.4780, Train Accuracy: 46.95%

Test Loss: 1.5009, Test Accuracy: 46.23%

Learning Rate: 0.004122

Epoch: 29, Train Loss: 1.4687, Train Accuracy: 47.29%

Epoch: 29, Test Loss: 1.4831, Test Accuracy: 47.21%

End of Epoch 29:

Privacy Budget: Epsilon = 2.2420

Train Loss: 1.4687, Train Accuracy: 47.29%

Test Loss: 1.4831, Test Accuracy: 47.21%

Learning Rate: 0.003819

Epoch: 30, Train Loss: 1.4746, Train Accuracy: 47.52%

Epoch: 30, Test Loss: 1.4844, Test Accuracy: 46.75%

Averaging model parameters...
End of Epoch 30:

Privacy Budget: Epsilon = 2.2823

Train Loss: 1.4746, Train Accuracy: 47.52%

Test Loss: 1.4844, Test Accuracy: 46.75%

Learning Rate: 0.003520

Epoch: 31, Train Loss: 1.4749, Train Accuracy: 47.28%

Epoch: 31, Test Loss: 1.4866, Test Accuracy: 46.62%

End of Epoch 31:

Privacy Budget: Epsilon = 2.3222

Train Loss: 1.4749, Train Accuracy: 47.28%

Test Loss: 1.4866, Test Accuracy: 46.62%

Learning Rate: 0.003228

Epoch: 32, Train Loss: 1.4650, Train Accuracy: 47.88%

Epoch: 32, Test Loss: 1.4752, Test Accuracy: 47.08%

End of Epoch 32:

Privacy Budget: Epsilon = 2.3614

Train Loss: 1.4650, Train Accuracy: 47.88%

Test Loss: 1.4752, Test Accuracy: 47.08%

Learning Rate: 0.002942

Epoch: 33, Train Loss: 1.4634, Train Accuracy: 48.01%

Epoch: 33, Test Loss: 1.4939, Test Accuracy: 47.18%

End of Epoch 33:

Privacy Budget: Epsilon = 2.4002

Train Loss: 1.4634, Train Accuracy: 48.01%

Test Loss: 1.4939, Test Accuracy: 47.18%

Learning Rate: 0.002665

Epoch: 34, Train Loss: 1.4659, Train Accuracy: 48.17%

Epoch: 34, Test Loss: 1.4857, Test Accuracy: 47.99%

End of Epoch 34:

Privacy Budget: Epsilon = 2.4385

Train Loss: 1.4659, Train Accuracy: 48.17%

Test Loss: 1.4857, Test Accuracy: 47.99%

Learning Rate: 0.002398

Epoch: 35, Train Loss: 1.4711, Train Accuracy: 48.07%

Epoch: 35, Test Loss: 1.4840, Test Accuracy: 47.46%

End of Epoch 35:

Privacy Budget: Epsilon = 2.4763

Train Loss: 1.4711, Train Accuracy: 48.07%

Test Loss: 1.4840, Test Accuracy: 47.46%

Learning Rate: 0.002140

Epoch: 36, Train Loss: 1.4639, Train Accuracy: 48.46%

Epoch: 36, Test Loss: 1.4883, Test Accuracy: 47.31%

End of Epoch 36:

Privacy Budget: Epsilon = 2.5137

Train Loss: 1.4639, Train Accuracy: 48.46%

Test Loss: 1.4883, Test Accuracy: 47.31%

Learning Rate: 0.001895

Epoch: 37, Train Loss: 1.4601, Train Accuracy: 48.70%

Epoch: 37, Test Loss: 1.4865, Test Accuracy: 47.75%

End of Epoch 37:

Privacy Budget: Epsilon = 2.5507

Train Loss: 1.4601, Train Accuracy: 48.70%

Test Loss: 1.4865, Test Accuracy: 47.75%

Learning Rate: 0.001661

Epoch: 38, Train Loss: 1.4631, Train Accuracy: 48.54%

Epoch: 38, Test Loss: 1.4774, Test Accuracy: 48.44%

End of Epoch 38:

Privacy Budget: Epsilon = 2.5872

Train Loss: 1.4631, Train Accuracy: 48.54%

Test Loss: 1.4774, Test Accuracy: 48.44%

Learning Rate: 0.001442

Epoch: 39, Train Loss: 1.4624, Train Accuracy: 48.49%

Epoch: 39, Test Loss: 1.4780, Test Accuracy: 47.67%

End of Epoch 39:

Privacy Budget: Epsilon = 2.6233

Train Loss: 1.4624, Train Accuracy: 48.49%

Test Loss: 1.4780, Test Accuracy: 47.67%

Learning Rate: 0.001236

Epoch: 40, Train Loss: 1.4651, Train Accuracy: 48.40%

Epoch: 40, Test Loss: 1.4861, Test Accuracy: 48.08%

Averaging model parameters...
End of Epoch 40:

Privacy Budget: Epsilon = 2.6591

Train Loss: 1.4651, Train Accuracy: 48.40%

Test Loss: 1.4861, Test Accuracy: 48.08%

Learning Rate: 0.001045

Epoch: 41, Train Loss: 1.4616, Train Accuracy: 48.73%

Epoch: 41, Test Loss: 1.4785, Test Accuracy: 48.45%

End of Epoch 41:

Privacy Budget: Epsilon = 2.6945

Train Loss: 1.4616, Train Accuracy: 48.73%

Test Loss: 1.4785, Test Accuracy: 48.45%

Learning Rate: 0.000871

Epoch: 42, Train Loss: 1.4584, Train Accuracy: 48.65%

Epoch: 42, Test Loss: 1.4870, Test Accuracy: 48.02%

End of Epoch 42:

Privacy Budget: Epsilon = 2.7295

Train Loss: 1.4584, Train Accuracy: 48.65%

Test Loss: 1.4870, Test Accuracy: 48.02%

Learning Rate: 0.000712

Epoch: 43, Train Loss: 1.4639, Train Accuracy: 48.74%

Epoch: 43, Test Loss: 1.4808, Test Accuracy: 48.51%

End of Epoch 43:

Privacy Budget: Epsilon = 2.7642

Train Loss: 1.4639, Train Accuracy: 48.74%

Test Loss: 1.4808, Test Accuracy: 48.51%

Learning Rate: 0.000571

Epoch: 44, Train Loss: 1.4569, Train Accuracy: 48.59%

Epoch: 44, Test Loss: 1.4850, Test Accuracy: 48.30%

End of Epoch 44:

Privacy Budget: Epsilon = 2.7986

Train Loss: 1.4569, Train Accuracy: 48.59%

Test Loss: 1.4850, Test Accuracy: 48.30%

Learning Rate: 0.000448

Epoch: 45, Train Loss: 1.4567, Train Accuracy: 48.92%

Epoch: 45, Test Loss: 1.4794, Test Accuracy: 47.93%

End of Epoch 45:

Privacy Budget: Epsilon = 2.8327

Train Loss: 1.4567, Train Accuracy: 48.92%

Test Loss: 1.4794, Test Accuracy: 47.93%

Learning Rate: 0.000342

Epoch: 46, Train Loss: 1.4549, Train Accuracy: 48.82%

Epoch: 46, Test Loss: 1.4790, Test Accuracy: 48.31%

End of Epoch 46:

Privacy Budget: Epsilon = 2.8664

Train Loss: 1.4549, Train Accuracy: 48.82%

Test Loss: 1.4790, Test Accuracy: 48.31%

Learning Rate: 0.000256

Epoch: 47, Train Loss: 1.4617, Train Accuracy: 48.86%

Epoch: 47, Test Loss: 1.4842, Test Accuracy: 48.11%

End of Epoch 47:

Privacy Budget: Epsilon = 2.8998

Train Loss: 1.4617, Train Accuracy: 48.86%

Test Loss: 1.4842, Test Accuracy: 48.11%

Learning Rate: 0.000188

Epoch: 48, Train Loss: 1.4571, Train Accuracy: 48.82%

Epoch: 48, Test Loss: 1.4810, Test Accuracy: 48.47%

End of Epoch 48:

Privacy Budget: Epsilon = 2.9330

Train Loss: 1.4571, Train Accuracy: 48.82%

Test Loss: 1.4810, Test Accuracy: 48.47%

Learning Rate: 0.000139


In [ ]:
import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


LR=0.1- GROUPED GRAD CLIPPING, PARAM AVERAGING, LR SCHEDULER, BATCH SIZE=256

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]


def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)


# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')

In [ ]:
import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_accuracy_vals)+1), train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(range(1, len(test_accuracy_vals)+1), test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_loss_vals)+1), train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(range(1, len(test_loss_vals)+1), test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


LR=0.1, batch_size= 128---- LR_batch_dec

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]


def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)


# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')


import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


Epoch: 1, Train Loss: 1.9180, Train Accuracy: 26.37%

Epoch: 1, Test Loss: 1.8123, Test Accuracy: 28.80%

End of Epoch 1:

Privacy Budget: Epsilon = 0.6033

Train Loss: 1.9180, Train Accuracy: 26.37%

Test Loss: 1.8123, Test Accuracy: 28.80%

Learning Rate: 0.099901

Epoch: 2, Train Loss: 1.7807, Train Accuracy: 32.29%

Epoch: 2, Test Loss: 1.7698, Test Accuracy: 32.87%

End of Epoch 2:

Privacy Budget: Epsilon = 0.7433

Train Loss: 1.7807, Train Accuracy: 32.29%

Test Loss: 1.7698, Test Accuracy: 32.87%

Learning Rate: 0.099606

Epoch: 3, Train Loss: 1.7819, Train Accuracy: 32.29%

Epoch: 3, Test Loss: 1.7588, Test Accuracy: 33.50%

End of Epoch 3:

Privacy Budget: Epsilon = 0.8495

Train Loss: 1.7819, Train Accuracy: 32.29%

Test Loss: 1.7588, Test Accuracy: 33.50%

Learning Rate: 0.099115

Epoch: 4, Train Loss: 1.7699, Train Accuracy: 32.86%

Epoch: 4, Test Loss: 1.8094, Test Accuracy: 32.56%

End of Epoch 4:

Privacy Budget: Epsilon = 0.9406

Train Loss: 1.7699, Train Accuracy: 32.86%

Test Loss: 1.8094, Test Accuracy: 32.56%

Learning Rate: 0.098431

Epoch: 5, Train Loss: 1.8145, Train Accuracy: 31.76%

Epoch: 5, Test Loss: 1.8102, Test Accuracy: 31.65%

End of Epoch 5:

Privacy Budget: Epsilon = 1.0225

Train Loss: 1.8145, Train Accuracy: 31.76%

Test Loss: 1.8102, Test Accuracy: 31.65%

Learning Rate: 0.097555

Epoch: 6, Train Loss: 1.8232, Train Accuracy: 30.63%

Epoch: 6, Test Loss: 1.8604, Test Accuracy: 31.77%

End of Epoch 6:

Privacy Budget: Epsilon = 1.0981

Train Loss: 1.8232, Train Accuracy: 30.63%

Test Loss: 1.8604, Test Accuracy: 31.77%

Learning Rate: 0.096492

Epoch: 7, Train Loss: 1.8230, Train Accuracy: 31.23%

Epoch: 7, Test Loss: 1.7955, Test Accuracy: 33.30%

End of Epoch 7:

Privacy Budget: Epsilon = 1.1690

Train Loss: 1.8230, Train Accuracy: 31.23%

Test Loss: 1.7955, Test Accuracy: 33.30%

Learning Rate: 0.095246

Epoch: 8, Train Loss: 1.8380, Train Accuracy: 31.19%

Epoch: 8, Test Loss: 1.7774, Test Accuracy: 33.16%

End of Epoch 8:

Privacy Budget: Epsilon = 1.2360

Train Loss: 1.8380, Train Accuracy: 31.19%

Test Loss: 1.7774, Test Accuracy: 33.16%

Learning Rate: 0.093822

Epoch: 9, Train Loss: 1.8463, Train Accuracy: 30.99%

Epoch: 9, Test Loss: 1.7571, Test Accuracy: 32.41%

End of Epoch 9:

Privacy Budget: Epsilon = 1.2998

Train Loss: 1.8463, Train Accuracy: 30.99%

Test Loss: 1.7571, Test Accuracy: 32.41%

Learning Rate: 0.092224

Epoch: 10, Train Loss: 1.8464, Train Accuracy: 30.75%

Epoch: 10, Test Loss: 1.7636, Test Accuracy: 32.50%

Averaging model parameters...
End of Epoch 10:

Privacy Budget: Epsilon = 1.3610

Train Loss: 1.8464, Train Accuracy: 30.75%

Test Loss: 1.7636, Test Accuracy: 32.50%

Learning Rate: 0.090460

Epoch: 11, Train Loss: 1.8515, Train Accuracy: 31.07%

Epoch: 11, Test Loss: 1.9341, Test Accuracy: 28.63%

End of Epoch 11:

Privacy Budget: Epsilon = 1.4199

Train Loss: 1.8515, Train Accuracy: 31.07%

Test Loss: 1.9341, Test Accuracy: 28.63%

Learning Rate: 0.088537

Epoch: 12, Train Loss: 1.8476, Train Accuracy: 30.44%

Epoch: 12, Test Loss: 1.8382, Test Accuracy: 31.64%

End of Epoch 12:

Privacy Budget: Epsilon = 1.4767

Train Loss: 1.8476, Train Accuracy: 30.44%

Test Loss: 1.8382, Test Accuracy: 31.64%

Learning Rate: 0.086462

Epoch: 13, Train Loss: 1.8858, Train Accuracy: 29.93%

Epoch: 13, Test Loss: 1.8415, Test Accuracy: 31.81%

End of Epoch 13:

Privacy Budget: Epsilon = 1.5316

Train Loss: 1.8858, Train Accuracy: 29.93%

Test Loss: 1.8415, Test Accuracy: 31.81%

Learning Rate: 0.084243

Epoch: 14, Train Loss: 1.8735, Train Accuracy: 30.34%

Epoch: 14, Test Loss: 1.8017, Test Accuracy: 31.69%

End of Epoch 14:

Privacy Budget: Epsilon = 1.5850

Train Loss: 1.8735, Train Accuracy: 30.34%

Test Loss: 1.8017, Test Accuracy: 31.69%

Learning Rate: 0.081889

Epoch: 15, Train Loss: 1.8551, Train Accuracy: 30.87%

Epoch: 15, Test Loss: 1.9331, Test Accuracy: 29.93%

End of Epoch 15:

Privacy Budget: Epsilon = 1.6369

Train Loss: 1.8551, Train Accuracy: 30.87%

Test Loss: 1.9331, Test Accuracy: 29.93%

Learning Rate: 0.079410

Epoch: 16, Train Loss: 1.8570, Train Accuracy: 30.68%

Epoch: 16, Test Loss: 1.7779, Test Accuracy: 32.82%

End of Epoch 16:

Privacy Budget: Epsilon = 1.6874

Train Loss: 1.8570, Train Accuracy: 30.68%

Test Loss: 1.7779, Test Accuracy: 32.82%

Learning Rate: 0.076815

Epoch: 17, Train Loss: 1.8527, Train Accuracy: 30.49%

Epoch: 17, Test Loss: 1.8579, Test Accuracy: 32.33%

End of Epoch 17:

Privacy Budget: Epsilon = 1.7367

Train Loss: 1.8527, Train Accuracy: 30.49%

Test Loss: 1.8579, Test Accuracy: 32.33%

Learning Rate: 0.074114

Epoch: 18, Train Loss: 1.8671, Train Accuracy: 30.63%

Epoch: 18, Test Loss: 1.8824, Test Accuracy: 30.37%

End of Epoch 18:

Privacy Budget: Epsilon = 1.7849

Train Loss: 1.8671, Train Accuracy: 30.63%

Test Loss: 1.8824, Test Accuracy: 30.37%

Learning Rate: 0.071318

Epoch: 19, Train Loss: 1.8510, Train Accuracy: 31.60%

Epoch: 19, Test Loss: 1.8019, Test Accuracy: 33.34%

End of Epoch 19:

Privacy Budget: Epsilon = 1.8320

Train Loss: 1.8510, Train Accuracy: 31.60%

Test Loss: 1.8019, Test Accuracy: 33.34%

Learning Rate: 0.068438

Epoch: 20, Train Loss: 1.8582, Train Accuracy: 31.57%

Epoch: 20, Test Loss: 1.8466, Test Accuracy: 31.46%

Averaging model parameters...
End of Epoch 20:

Privacy Budget: Epsilon = 1.8782

Train Loss: 1.8582, Train Accuracy: 31.57%

Test Loss: 1.8466, Test Accuracy: 31.46%

Learning Rate: 0.065485

Epoch: 21, Train Loss: 1.8447, Train Accuracy: 31.48%

Epoch: 21, Test Loss: 2.1607, Test Accuracy: 26.61%

End of Epoch 21:

Privacy Budget: Epsilon = 1.9235

Train Loss: 1.8447, Train Accuracy: 31.48%

Test Loss: 2.1607, Test Accuracy: 26.61%

Learning Rate: 0.062472

Epoch: 22, Train Loss: 1.8450, Train Accuracy: 31.48%

Epoch: 22, Test Loss: 1.7801, Test Accuracy: 32.73%

End of Epoch 22:

Privacy Budget: Epsilon = 1.9678

Train Loss: 1.8450, Train Accuracy: 31.48%

Test Loss: 1.7801, Test Accuracy: 32.73%

Learning Rate: 0.059410

Epoch: 23, Train Loss: 1.8523, Train Accuracy: 31.77%

Epoch: 23, Test Loss: 1.9541, Test Accuracy: 28.76%

End of Epoch 23:

Privacy Budget: Epsilon = 2.0114

Train Loss: 1.8523, Train Accuracy: 31.77%

Test Loss: 1.9541, Test Accuracy: 28.76%

Learning Rate: 0.056310

Epoch: 24, Train Loss: 1.8268, Train Accuracy: 31.99%

Epoch: 24, Test Loss: 1.8233, Test Accuracy: 32.06%

End of Epoch 24:

Privacy Budget: Epsilon = 2.0543

Train Loss: 1.8268, Train Accuracy: 31.99%

Test Loss: 1.8233, Test Accuracy: 32.06%

Learning Rate: 0.053186

Epoch: 25, Train Loss: 1.8165, Train Accuracy: 32.44%

Epoch: 25, Test Loss: 1.8189, Test Accuracy: 32.27%

End of Epoch 25:

Privacy Budget: Epsilon = 2.0964

Train Loss: 1.8165, Train Accuracy: 32.44%

Test Loss: 1.8189, Test Accuracy: 32.27%

Learning Rate: 0.050050

Epoch: 26, Train Loss: 1.8090, Train Accuracy: 32.74%

Epoch: 26, Test Loss: 1.8026, Test Accuracy: 33.53%

End of Epoch 26:

Privacy Budget: Epsilon = 2.1379

Train Loss: 1.8090, Train Accuracy: 32.74%

Test Loss: 1.8026, Test Accuracy: 33.53%

Learning Rate: 0.046914

Epoch: 27, Train Loss: 1.8241, Train Accuracy: 32.66%

Epoch: 27, Test Loss: 1.8548, Test Accuracy: 32.61%

End of Epoch 27:

Privacy Budget: Epsilon = 2.1787

Train Loss: 1.8241, Train Accuracy: 32.66%

Test Loss: 1.8548, Test Accuracy: 32.61%

Learning Rate: 0.043790

Epoch: 28, Train Loss: 1.8100, Train Accuracy: 32.84%

Epoch: 28, Test Loss: 1.7679, Test Accuracy: 35.34%

End of Epoch 28:

Privacy Budget: Epsilon = 2.2190

Train Loss: 1.8100, Train Accuracy: 32.84%

Test Loss: 1.7679, Test Accuracy: 35.34%

Learning Rate: 0.040690

Epoch: 29, Train Loss: 1.8250, Train Accuracy: 32.72%

Epoch: 29, Test Loss: 1.8393, Test Accuracy: 33.23%

End of Epoch 29:

Privacy Budget: Epsilon = 2.2586

Train Loss: 1.8250, Train Accuracy: 32.72%

Test Loss: 1.8393, Test Accuracy: 33.23%

Learning Rate: 0.037628

Epoch: 30, Train Loss: 1.8170, Train Accuracy: 32.97%

Epoch: 30, Test Loss: 1.8097, Test Accuracy: 34.50%

Averaging model parameters...
End of Epoch 30:

Privacy Budget: Epsilon = 2.2978

Train Loss: 1.8170, Train Accuracy: 32.97%

Test Loss: 1.8097, Test Accuracy: 34.50%

Learning Rate: 0.034615

Epoch: 31, Train Loss: 1.8205, Train Accuracy: 33.29%

Epoch: 31, Test Loss: 1.7701, Test Accuracy: 33.79%

End of Epoch 31:

Privacy Budget: Epsilon = 2.3364

Train Loss: 1.8205, Train Accuracy: 33.29%

Test Loss: 1.7701, Test Accuracy: 33.79%

Learning Rate: 0.031662

Epoch: 32, Train Loss: 1.8012, Train Accuracy: 33.87%

Epoch: 32, Test Loss: 1.8103, Test Accuracy: 32.51%

End of Epoch 32:

Privacy Budget: Epsilon = 2.3745

Train Loss: 1.8012, Train Accuracy: 33.87%

Test Loss: 1.8103, Test Accuracy: 32.51%

Learning Rate: 0.028782

Epoch: 33, Train Loss: 1.7929, Train Accuracy: 34.10%

Epoch: 33, Test Loss: 1.7582, Test Accuracy: 35.62%

End of Epoch 33:

Privacy Budget: Epsilon = 2.4121

Train Loss: 1.7929, Train Accuracy: 34.10%

Test Loss: 1.7582, Test Accuracy: 35.62%

Learning Rate: 0.025986

Epoch: 34, Train Loss: 1.7837, Train Accuracy: 34.75%

Epoch: 34, Test Loss: 1.7791, Test Accuracy: 35.37%

End of Epoch 34:

Privacy Budget: Epsilon = 2.4492

Train Loss: 1.7837, Train Accuracy: 34.75%

Test Loss: 1.7791, Test Accuracy: 35.37%

Learning Rate: 0.023285

Epoch: 35, Train Loss: 1.7857, Train Accuracy: 34.93%

Epoch: 35, Test Loss: 1.7411, Test Accuracy: 35.55%

End of Epoch 35:

Privacy Budget: Epsilon = 2.4860

Train Loss: 1.7857, Train Accuracy: 34.93%

Test Loss: 1.7411, Test Accuracy: 35.55%

Learning Rate: 0.020690

Epoch: 36, Train Loss: 1.7746, Train Accuracy: 35.35%

Epoch: 36, Test Loss: 1.7544, Test Accuracy: 36.21%

End of Epoch 36:

Privacy Budget: Epsilon = 2.5223

Train Loss: 1.7746, Train Accuracy: 35.35%

Test Loss: 1.7544, Test Accuracy: 36.21%

Learning Rate: 0.018211

Epoch: 37, Train Loss: 1.7757, Train Accuracy: 34.88%

Epoch: 37, Test Loss: 1.7911, Test Accuracy: 35.47%

End of Epoch 37:

Privacy Budget: Epsilon = 2.5582

Train Loss: 1.7757, Train Accuracy: 34.88%

Test Loss: 1.7911, Test Accuracy: 35.47%

Learning Rate: 0.015857

Epoch: 38, Train Loss: 1.7627, Train Accuracy: 35.75%

Epoch: 38, Test Loss: 1.7289, Test Accuracy: 37.60%

End of Epoch 38:

Privacy Budget: Epsilon = 2.5937

Train Loss: 1.7627, Train Accuracy: 35.75%

Test Loss: 1.7289, Test Accuracy: 37.60%

Learning Rate: 0.013638

Epoch: 39, Train Loss: 1.7612, Train Accuracy: 36.12%

Epoch: 39, Test Loss: 1.7700, Test Accuracy: 36.82%

End of Epoch 39:

Privacy Budget: Epsilon = 2.6289

Train Loss: 1.7612, Train Accuracy: 36.12%

Test Loss: 1.7700, Test Accuracy: 36.82%

Learning Rate: 0.011563

Epoch: 40, Train Loss: 1.7469, Train Accuracy: 36.44%

Epoch: 40, Test Loss: 1.7467, Test Accuracy: 37.21%

Averaging model parameters...
End of Epoch 40:

Privacy Budget: Epsilon = 2.6637

Train Loss: 1.7469, Train Accuracy: 36.44%

Test Loss: 1.7467, Test Accuracy: 37.21%

Learning Rate: 0.009640

Epoch: 41, Train Loss: 1.7467, Train Accuracy: 36.72%

Epoch: 41, Test Loss: 1.7222, Test Accuracy: 38.28%

End of Epoch 41:

Privacy Budget: Epsilon = 2.6981

Train Loss: 1.7467, Train Accuracy: 36.72%

Test Loss: 1.7222, Test Accuracy: 38.28%

Learning Rate: 0.007876

Epoch: 42, Train Loss: 1.7384, Train Accuracy: 37.00%

Epoch: 42, Test Loss: 1.7755, Test Accuracy: 37.00%

End of Epoch 42:

Privacy Budget: Epsilon = 2.7322

Train Loss: 1.7384, Train Accuracy: 37.00%

Test Loss: 1.7755, Test Accuracy: 37.00%

Learning Rate: 0.006278

Epoch: 43, Train Loss: 1.7228, Train Accuracy: 37.70%

Epoch: 43, Test Loss: 1.7159, Test Accuracy: 38.25%

End of Epoch 43:

Privacy Budget: Epsilon = 2.7660

Train Loss: 1.7228, Train Accuracy: 37.70%

Test Loss: 1.7159, Test Accuracy: 38.25%

Learning Rate: 0.004854

Epoch: 44, Train Loss: 1.7181, Train Accuracy: 37.82%

Epoch: 44, Test Loss: 1.7201, Test Accuracy: 38.22%

End of Epoch 44:

Privacy Budget: Epsilon = 2.7995

Train Loss: 1.7181, Train Accuracy: 37.82%

Test Loss: 1.7201, Test Accuracy: 38.22%

Learning Rate: 0.003608

Epoch: 45, Train Loss: 1.7198, Train Accuracy: 37.90%

Epoch: 45, Test Loss: 1.7203, Test Accuracy: 38.46%

End of Epoch 45:

Privacy Budget: Epsilon = 2.8326

Train Loss: 1.7198, Train Accuracy: 37.90%

Test Loss: 1.7203, Test Accuracy: 38.46%

Learning Rate: 0.002545

Epoch: 46, Train Loss: 1.7162, Train Accuracy: 38.16%

Epoch: 46, Test Loss: 1.7223, Test Accuracy: 38.98%

End of Epoch 46:

Privacy Budget: Epsilon = 2.8655

Train Loss: 1.7162, Train Accuracy: 38.16%

Test Loss: 1.7223, Test Accuracy: 38.98%

Learning Rate: 0.001669

Epoch: 47, Train Loss: 1.7230, Train Accuracy: 38.31%

Epoch: 47, Test Loss: 1.7128, Test Accuracy: 39.18%

End of Epoch 47:

Privacy Budget: Epsilon = 2.8981

Train Loss: 1.7230, Train Accuracy: 38.31%

Test Loss: 1.7128, Test Accuracy: 39.18%

Learning Rate: 0.000985

Epoch: 48, Train Loss: 1.7080, Train Accuracy: 38.59%

Epoch: 48, Test Loss: 1.7153, Test Accuracy: 38.81%

End of Epoch 48:

Privacy Budget: Epsilon = 2.9304

Train Loss: 1.7080, Train Accuracy: 38.59%

Test Loss: 1.7153, Test Accuracy: 38.81%

Learning Rate: 0.000494

Epoch: 49, Train Loss: 1.7177, Train Accuracy: 38.17%

Epoch: 49, Test Loss: 1.7103, Test Accuracy: 39.13%

End of Epoch 49:

Privacy Budget: Epsilon = 2.9625

Train Loss: 1.7177, Train Accuracy: 38.17%

Test Loss: 1.7103, Test Accuracy: 39.13%

Learning Rate: 0.000199

LEARNING RATE=0.001

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]


def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=8, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)


# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')

Epoch: 1, Train Loss: 2.2874, Train Accuracy: 14.78%

Epoch: 1, Test Loss: 2.2334, Test Accuracy: 18.96%

End of Epoch 1:

Privacy Budget: Epsilon = 0.4951

Train Loss: 2.2874, Train Accuracy: 14.78%

Test Loss: 2.2334, Test Accuracy: 18.96%

Learning Rate: 0.000999

Epoch: 2, Train Loss: 2.1694, Train Accuracy: 19.82%

Epoch: 2, Test Loss: 2.1123, Test Accuracy: 19.66%

End of Epoch 2:

Privacy Budget: Epsilon = 0.6385

Train Loss: 2.1694, Train Accuracy: 19.82%

Test Loss: 2.1123, Test Accuracy: 19.66%

Learning Rate: 0.000996

Epoch: 3, Train Loss: 2.0807, Train Accuracy: 20.35%

Epoch: 3, Test Loss: 2.0540, Test Accuracy: 21.50%

End of Epoch 3:

Privacy Budget: Epsilon = 0.7527

Train Loss: 2.0807, Train Accuracy: 20.35%

Test Loss: 2.0540, Test Accuracy: 21.50%

Learning Rate: 0.000992

Epoch: 4, Train Loss: 2.0389, Train Accuracy: 22.25%

Epoch: 4, Test Loss: 2.0201, Test Accuracy: 23.56%

End of Epoch 4:

Privacy Budget: Epsilon = 0.8518

Train Loss: 2.0389, Train Accuracy: 22.25%

Test Loss: 2.0201, Test Accuracy: 23.56%

Learning Rate: 0.000986

Epoch: 5, Train Loss: 2.0073, Train Accuracy: 24.25%

Epoch: 5, Test Loss: 1.9902, Test Accuracy: 25.17%

End of Epoch 5:

Privacy Budget: Epsilon = 0.9411

Train Loss: 2.0073, Train Accuracy: 24.25%

Test Loss: 1.9902, Test Accuracy: 25.17%

Learning Rate: 0.000978

Epoch: 6, Train Loss: 1.9798, Train Accuracy: 25.92%

Epoch: 6, Test Loss: 1.9604, Test Accuracy: 26.13%

End of Epoch 6:

Privacy Budget: Epsilon = 1.0231

Train Loss: 1.9798, Train Accuracy: 25.92%

Test Loss: 1.9604, Test Accuracy: 26.13%

Learning Rate: 0.000968

Epoch: 7, Train Loss: 1.9463, Train Accuracy: 26.69%

Epoch: 7, Test Loss: 1.9327, Test Accuracy: 28.26%

End of Epoch 7:

Privacy Budget: Epsilon = 1.0995

Train Loss: 1.9463, Train Accuracy: 26.69%

Test Loss: 1.9327, Test Accuracy: 28.26%

Learning Rate: 0.000957

Epoch: 8, Train Loss: 1.9198, Train Accuracy: 28.85%

Epoch: 8, Test Loss: 1.9055, Test Accuracy: 29.92%

End of Epoch 8:

Privacy Budget: Epsilon = 1.1715

Train Loss: 1.9198, Train Accuracy: 28.85%

Test Loss: 1.9055, Test Accuracy: 29.92%

Learning Rate: 0.000944

Epoch: 9, Train Loss: 1.8999, Train Accuracy: 30.20%

Epoch: 9, Test Loss: 1.8823, Test Accuracy: 31.61%

End of Epoch 9:

Privacy Budget: Epsilon = 1.2397

Train Loss: 1.8999, Train Accuracy: 30.20%

Test Loss: 1.8823, Test Accuracy: 31.61%

Learning Rate: 0.000930

Epoch: 10, Train Loss: 1.8697, Train Accuracy: 31.17%

Epoch: 10, Test Loss: 1.8622, Test Accuracy: 31.11%

Averaging model parameters...
End of Epoch 10:

Privacy Budget: Epsilon = 1.3049

Train Loss: 1.8697, Train Accuracy: 31.17%

Test Loss: 1.8622, Test Accuracy: 31.11%

Learning Rate: 0.000914

Epoch: 11, Train Loss: 1.8475, Train Accuracy: 31.35%

Epoch: 11, Test Loss: 1.8435, Test Accuracy: 32.31%

End of Epoch 11:

Privacy Budget: Epsilon = 1.3673

Train Loss: 1.8475, Train Accuracy: 31.35%

Test Loss: 1.8435, Test Accuracy: 32.31%

Learning Rate: 0.000897

Epoch: 12, Train Loss: 1.8368, Train Accuracy: 31.87%

Epoch: 12, Test Loss: 1.8273, Test Accuracy: 32.48%

End of Epoch 12:

Privacy Budget: Epsilon = 1.4274

Train Loss: 1.8368, Train Accuracy: 31.87%

Test Loss: 1.8273, Test Accuracy: 32.48%

Learning Rate: 0.000878

Epoch: 13, Train Loss: 1.8245, Train Accuracy: 32.49%

Epoch: 13, Test Loss: 1.8146, Test Accuracy: 33.37%

End of Epoch 13:

Privacy Budget: Epsilon = 1.4854

Train Loss: 1.8245, Train Accuracy: 32.49%

Test Loss: 1.8146, Test Accuracy: 33.37%

Learning Rate: 0.000858

Epoch: 14, Train Loss: 1.8103, Train Accuracy: 32.94%

Epoch: 14, Test Loss: 1.8014, Test Accuracy: 33.62%

End of Epoch 14:

Privacy Budget: Epsilon = 1.5415

Train Loss: 1.8103, Train Accuracy: 32.94%

Test Loss: 1.8014, Test Accuracy: 33.62%

Learning Rate: 0.000837

Epoch: 15, Train Loss: 1.7910, Train Accuracy: 33.99%

Epoch: 15, Test Loss: 1.7915, Test Accuracy: 33.67%

End of Epoch 15:

Privacy Budget: Epsilon = 1.5959

Train Loss: 1.7910, Train Accuracy: 33.99%

Test Loss: 1.7915, Test Accuracy: 33.67%

Learning Rate: 0.000815

Epoch: 16, Train Loss: 1.7821, Train Accuracy: 33.77%

Epoch: 16, Test Loss: 1.7806, Test Accuracy: 34.00%

End of Epoch 16:

Privacy Budget: Epsilon = 1.6489

Train Loss: 1.7821, Train Accuracy: 33.77%

Test Loss: 1.7806, Test Accuracy: 34.00%

Learning Rate: 0.000791

Epoch: 17, Train Loss: 1.7722, Train Accuracy: 34.14%

Epoch: 17, Test Loss: 1.7714, Test Accuracy: 34.34%

End of Epoch 17:

Privacy Budget: Epsilon = 1.7004

Train Loss: 1.7722, Train Accuracy: 34.14%

Test Loss: 1.7714, Test Accuracy: 34.34%

Learning Rate: 0.000767

Epoch: 18, Train Loss: 1.7644, Train Accuracy: 34.03%

Epoch: 18, Test Loss: 1.7621, Test Accuracy: 34.72%

End of Epoch 18:

Privacy Budget: Epsilon = 1.7507

Train Loss: 1.7644, Train Accuracy: 34.03%

Test Loss: 1.7621, Test Accuracy: 34.72%

Learning Rate: 0.000742

Epoch: 19, Train Loss: 1.7632, Train Accuracy: 33.96%

Epoch: 19, Test Loss: 1.7560, Test Accuracy: 34.70%

End of Epoch 19:

Privacy Budget: Epsilon = 1.7998

Train Loss: 1.7632, Train Accuracy: 33.96%

Test Loss: 1.7560, Test Accuracy: 34.70%

Learning Rate: 0.000716

Epoch: 20, Train Loss: 1.7543, Train Accuracy: 34.75%

Epoch: 20, Test Loss: 1.7479, Test Accuracy: 35.07%

Averaging model parameters...
End of Epoch 20:

Privacy Budget: Epsilon = 1.8479

Train Loss: 1.7543, Train Accuracy: 34.75%

Test Loss: 1.7479, Test Accuracy: 35.07%

Learning Rate: 0.000689

Epoch: 21, Train Loss: 1.7444, Train Accuracy: 34.54%

Epoch: 21, Test Loss: 1.7407, Test Accuracy: 35.30%

End of Epoch 21:

Privacy Budget: Epsilon = 1.8949

Train Loss: 1.7444, Train Accuracy: 34.54%

Test Loss: 1.7407, Test Accuracy: 35.30%

Learning Rate: 0.000662

Epoch: 22, Train Loss: 1.7266, Train Accuracy: 35.16%

Epoch: 22, Test Loss: 1.7373, Test Accuracy: 35.44%

End of Epoch 22:

Privacy Budget: Epsilon = 1.9410

Train Loss: 1.7266, Train Accuracy: 35.16%

Test Loss: 1.7373, Test Accuracy: 35.44%

Learning Rate: 0.000634

Epoch: 23, Train Loss: 1.7303, Train Accuracy: 35.08%

Epoch: 23, Test Loss: 1.7299, Test Accuracy: 35.60%

End of Epoch 23:

Privacy Budget: Epsilon = 1.9863

Train Loss: 1.7303, Train Accuracy: 35.08%

Test Loss: 1.7299, Test Accuracy: 35.60%

Learning Rate: 0.000606

Epoch: 24, Train Loss: 1.7229, Train Accuracy: 35.24%

Epoch: 24, Test Loss: 1.7259, Test Accuracy: 35.84%

End of Epoch 24:

Privacy Budget: Epsilon = 2.0307

Train Loss: 1.7229, Train Accuracy: 35.24%

Test Loss: 1.7259, Test Accuracy: 35.84%

Learning Rate: 0.000578

Epoch: 25, Train Loss: 1.7187, Train Accuracy: 35.56%

Epoch: 25, Test Loss: 1.7213, Test Accuracy: 36.13%

End of Epoch 25:

Privacy Budget: Epsilon = 2.0743

Train Loss: 1.7187, Train Accuracy: 35.56%

Test Loss: 1.7213, Test Accuracy: 36.13%

Learning Rate: 0.000550

Epoch: 26, Train Loss: 1.7124, Train Accuracy: 35.76%

Epoch: 26, Test Loss: 1.7165, Test Accuracy: 36.17%

End of Epoch 26:

Privacy Budget: Epsilon = 2.1172

Train Loss: 1.7124, Train Accuracy: 35.76%

Test Loss: 1.7165, Test Accuracy: 36.17%

Learning Rate: 0.000522

Epoch: 27, Train Loss: 1.7072, Train Accuracy: 35.95%

Epoch: 27, Test Loss: 1.7127, Test Accuracy: 36.23%

End of Epoch 27:

Privacy Budget: Epsilon = 2.1594

Train Loss: 1.7072, Train Accuracy: 35.95%

Test Loss: 1.7127, Test Accuracy: 36.23%

Learning Rate: 0.000494

Epoch: 28, Train Loss: 1.7044, Train Accuracy: 36.06%

Epoch: 28, Test Loss: 1.7094, Test Accuracy: 36.38%

End of Epoch 28:

Privacy Budget: Epsilon = 2.2010

Train Loss: 1.7044, Train Accuracy: 36.06%

Test Loss: 1.7094, Test Accuracy: 36.38%

Learning Rate: 0.000466

In [ ]:
import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import opacus
print(opacus.__version__)


LR=0.01, BATCH SIZE: 512

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]


def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)


# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')

import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


ADD DYNAMIC NOISE SCALING TO LR=0.01, BATCH SIZE=**256**

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

def update_noise_multiplier(loss, base_noise, max_noise, min_noise, scale_factor):
    """
    Dynamically update noise multiplier based on training loss.

    Args:
    - loss (float): Current training loss.
    - base_noise (float): Base noise multiplier.
    - max_noise (float): Maximum noise multiplier.
    - min_noise (float): Minimum noise multiplier.
    - scale_factor (float): Sensitivity of noise adjustment to loss.

    Returns:
    - float: Updated noise multiplier.
    """
    # Adjust noise dynamically (higher loss -> higher noise, lower loss -> lower noise)
    noise_multiplier = base_noise + scale_factor * loss
    return max(min_noise, min(noise_multiplier, max_noise))  # Clamp to [min_noise, max_noise]


def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)


# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Parameters for dynamic noise adjustment
base_noise = 1.0  # Initial noise multiplier
max_noise = 3.0   # Maximum noise allowed
min_noise = 0.5   # Minimum noise allowed
scale_factor = 0.1  # Scaling factor to adjust sensitivity


# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)
    # Dynamically adjust noise multiplier based on training loss
    new_noise = update_noise_multiplier(
        loss=train_loss,
        base_noise=base_noise,
        max_noise=max_noise,
        min_noise=min_noise,
        scale_factor=scale_factor
    )
    optimizer.noise_multiplier = new_noise  # Update the noise multiplier
    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f"Epoch {epoch}, Updated Noise Multiplier: {new_noise:.4f}\n")

    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')

import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


Files already downloaded and verified
Files already downloaded and verified
/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
WARNING:opacus.data_loader:Ignoring drop_last as it is not compatible with DPDataLoader.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
Using sigma=0.97412109375

Epoch: 1, Train Loss: 2.1153, Train Accuracy: 19.92%

Epoch: 1, Test Loss: 1.9779, Test Accuracy: 24.84%

End of Epoch 1:

Epoch 1, Updated Noise Multiplier: 1.2115

Privacy Budget: Epsilon = 0.4951

Train Loss: 2.1153, Train Accuracy: 19.92%

Test Loss: 1.9779, Test Accuracy: 24.84%

Learning Rate: 0.009990

Epoch: 2, Train Loss: 1.9210, Train Accuracy: 27.82%

Epoch: 2, Test Loss: 1.8645, Test Accuracy: 29.92%

End of Epoch 2:

Epoch 2, Updated Noise Multiplier: 1.1921

Privacy Budget: Epsilon = 0.5486

Train Loss: 1.9210, Train Accuracy: 27.82%

Test Loss: 1.8645, Test Accuracy: 29.92%

Learning Rate: 0.009961

Epoch: 3, Train Loss: 1.8224, Train Accuracy: 31.34%

Epoch: 3, Test Loss: 1.7874, Test Accuracy: 32.33%

End of Epoch 3:

Epoch 3, Updated Noise Multiplier: 1.1822

Privacy Budget: Epsilon = 0.6057

Train Loss: 1.8224, Train Accuracy: 31.34%

Test Loss: 1.7874, Test Accuracy: 32.33%

Learning Rate: 0.009912

Epoch: 4, Train Loss: 1.7728, Train Accuracy: 32.52%

Epoch: 4, Test Loss: 1.7578, Test Accuracy: 33.34%

End of Epoch 4:

Epoch 4, Updated Noise Multiplier: 1.1773

Privacy Budget: Epsilon = 0.6629

Train Loss: 1.7728, Train Accuracy: 32.52%

Test Loss: 1.7578, Test Accuracy: 33.34%

Learning Rate: 0.009844

Epoch: 5, Train Loss: 1.7492, Train Accuracy: 32.94%

Epoch: 5, Test Loss: 1.7329, Test Accuracy: 34.56%

End of Epoch 5:

Epoch 5, Updated Noise Multiplier: 1.1749

Privacy Budget: Epsilon = 0.7186

Train Loss: 1.7492, Train Accuracy: 32.94%

Test Loss: 1.7329, Test Accuracy: 34.56%

Learning Rate: 0.009758

Epoch: 6, Train Loss: 1.7304, Train Accuracy: 34.58%

Epoch: 6, Test Loss: 1.7299, Test Accuracy: 33.83%

End of Epoch 6:

Epoch 6, Updated Noise Multiplier: 1.1730

Privacy Budget: Epsilon = 0.7722

Train Loss: 1.7304, Train Accuracy: 34.58%

Test Loss: 1.7299, Test Accuracy: 33.83%

Learning Rate: 0.009652

Epoch: 7, Train Loss: 1.7146, Train Accuracy: 34.55%

Epoch: 7, Test Loss: 1.6956, Test Accuracy: 34.49%

End of Epoch 7:

Epoch 7, Updated Noise Multiplier: 1.1715

Privacy Budget: Epsilon = 0.8236

Train Loss: 1.7146, Train Accuracy: 34.55%

Test Loss: 1.6956, Test Accuracy: 34.49%

Learning Rate: 0.009529

Epoch: 8, Train Loss: 1.6959, Train Accuracy: 35.15%

Epoch: 8, Test Loss: 1.6836, Test Accuracy: 36.72%

End of Epoch 8:

Epoch 8, Updated Noise Multiplier: 1.1696

Privacy Budget: Epsilon = 0.8729

Train Loss: 1.6959, Train Accuracy: 35.15%

Test Loss: 1.6836, Test Accuracy: 36.72%

Learning Rate: 0.009388

Epoch: 9, Train Loss: 1.6780, Train Accuracy: 36.34%

Epoch: 9, Test Loss: 1.6821, Test Accuracy: 36.01%

End of Epoch 9:

Epoch 9, Updated Noise Multiplier: 1.1678

Privacy Budget: Epsilon = 0.9205

Train Loss: 1.6780, Train Accuracy: 36.34%

Test Loss: 1.6821, Test Accuracy: 36.01%

Learning Rate: 0.009229

Epoch: 10, Train Loss: 1.6630, Train Accuracy: 36.73%

Epoch: 10, Test Loss: 1.6590, Test Accuracy: 37.35%

Averaging model parameters...
End of Epoch 10:

Epoch 10, Updated Noise Multiplier: 1.1663

Privacy Budget: Epsilon = 0.9666

Train Loss: 1.6630, Train Accuracy: 36.73%

Test Loss: 1.6590, Test Accuracy: 37.35%

Learning Rate: 0.009055

Epoch: 11, Train Loss: 1.6456, Train Accuracy: 37.35%

Epoch: 11, Test Loss: 1.6543, Test Accuracy: 37.73%

End of Epoch 11:

Epoch 11, Updated Noise Multiplier: 1.1646

Privacy Budget: Epsilon = 1.0111

Train Loss: 1.6456, Train Accuracy: 37.35%

Test Loss: 1.6543, Test Accuracy: 37.73%

Learning Rate: 0.008864

Epoch: 12, Train Loss: 1.6464, Train Accuracy: 38.35%

Epoch: 12, Test Loss: 1.6420, Test Accuracy: 38.99%

End of Epoch 12:

Epoch 12, Updated Noise Multiplier: 1.1646

Privacy Budget: Epsilon = 1.0543

Train Loss: 1.6464, Train Accuracy: 38.35%

Test Loss: 1.6420, Test Accuracy: 38.99%

Learning Rate: 0.008658

Epoch: 13, Train Loss: 1.6373, Train Accuracy: 38.95%

Epoch: 13, Test Loss: 1.6222, Test Accuracy: 38.63%

End of Epoch 13:

Epoch 13, Updated Noise Multiplier: 1.1637

Privacy Budget: Epsilon = 1.0961

Train Loss: 1.6373, Train Accuracy: 38.95%

Test Loss: 1.6222, Test Accuracy: 38.63%

Learning Rate: 0.008439

Epoch: 14, Train Loss: 1.6161, Train Accuracy: 40.54%

Epoch: 14, Test Loss: 1.6037, Test Accuracy: 41.04%

End of Epoch 14:

Epoch 14, Updated Noise Multiplier: 1.1616

Privacy Budget: Epsilon = 1.1368

Train Loss: 1.6161, Train Accuracy: 40.54%

Test Loss: 1.6037, Test Accuracy: 41.04%

Learning Rate: 0.008205

Epoch: 15, Train Loss: 1.6033, Train Accuracy: 40.62%

Epoch: 15, Test Loss: 1.6085, Test Accuracy: 40.67%

End of Epoch 15:

Epoch 15, Updated Noise Multiplier: 1.1603

Privacy Budget: Epsilon = 1.1766

Train Loss: 1.6033, Train Accuracy: 40.62%

Test Loss: 1.6085, Test Accuracy: 40.67%

Learning Rate: 0.007960

Epoch: 16, Train Loss: 1.5768, Train Accuracy: 41.57%

Epoch: 16, Test Loss: 1.5976, Test Accuracy: 40.87%

End of Epoch 16:

Epoch 16, Updated Noise Multiplier: 1.1577

Privacy Budget: Epsilon = 1.2155

Train Loss: 1.5768, Train Accuracy: 41.57%

Test Loss: 1.5976, Test Accuracy: 40.87%

Learning Rate: 0.007702

Epoch: 17, Train Loss: 1.5654, Train Accuracy: 42.40%

Epoch: 17, Test Loss: 1.5654, Test Accuracy: 42.41%

End of Epoch 17:

Epoch 17, Updated Noise Multiplier: 1.1565

Privacy Budget: Epsilon = 1.2536

Train Loss: 1.5654, Train Accuracy: 42.40%

Test Loss: 1.5654, Test Accuracy: 42.41%

Learning Rate: 0.007435

Epoch: 18, Train Loss: 1.5519, Train Accuracy: 43.11%

Epoch: 18, Test Loss: 1.5509, Test Accuracy: 43.21%

End of Epoch 18:

Epoch 18, Updated Noise Multiplier: 1.1552

Privacy Budget: Epsilon = 1.2909

Train Loss: 1.5519, Train Accuracy: 43.11%

Test Loss: 1.5509, Test Accuracy: 43.21%

Learning Rate: 0.007158

Epoch: 19, Train Loss: 1.5315, Train Accuracy: 43.62%

Epoch: 19, Test Loss: 1.5392, Test Accuracy: 44.10%

End of Epoch 19:

Epoch 19, Updated Noise Multiplier: 1.1532

Privacy Budget: Epsilon = 1.3275

Train Loss: 1.5315, Train Accuracy: 43.62%

Test Loss: 1.5392, Test Accuracy: 44.10%

Learning Rate: 0.006872

Epoch: 20, Train Loss: 1.5292, Train Accuracy: 43.84%

Epoch: 20, Test Loss: 1.5533, Test Accuracy: 42.88%

Averaging model parameters...
End of Epoch 20:

Epoch 20, Updated Noise Multiplier: 1.1529

Privacy Budget: Epsilon = 1.3635

Train Loss: 1.5292, Train Accuracy: 43.84%

Test Loss: 1.5533, Test Accuracy: 42.88%

Learning Rate: 0.006580

Epoch: 21, Train Loss: 1.5175, Train Accuracy: 44.67%

Epoch: 21, Test Loss: 1.5302, Test Accuracy: 44.13%

End of Epoch 21:

Epoch 21, Updated Noise Multiplier: 1.1518

Privacy Budget: Epsilon = 1.3988

Train Loss: 1.5175, Train Accuracy: 44.67%

Test Loss: 1.5302, Test Accuracy: 44.13%

Learning Rate: 0.006281

Epoch: 22, Train Loss: 1.5118, Train Accuracy: 44.73%

Epoch: 22, Test Loss: 1.5117, Test Accuracy: 44.91%

End of Epoch 22:

Epoch 22, Updated Noise Multiplier: 1.1512

Privacy Budget: Epsilon = 1.4334

Train Loss: 1.5118, Train Accuracy: 44.73%

Test Loss: 1.5117, Test Accuracy: 44.91%

Learning Rate: 0.005978

Epoch: 23, Train Loss: 1.5013, Train Accuracy: 45.22%

Epoch: 23, Test Loss: 1.5121, Test Accuracy: 45.11%

End of Epoch 23:

Epoch 23, Updated Noise Multiplier: 1.1501

Privacy Budget: Epsilon = 1.4675

Train Loss: 1.5013, Train Accuracy: 45.22%

Test Loss: 1.5121, Test Accuracy: 45.11%

Learning Rate: 0.005670

Epoch: 24, Train Loss: 1.4869, Train Accuracy: 45.66%

Epoch: 24, Test Loss: 1.4892, Test Accuracy: 45.40%

End of Epoch 24:

Epoch 24, Updated Noise Multiplier: 1.1487

Privacy Budget: Epsilon = 1.5010

Train Loss: 1.4869, Train Accuracy: 45.66%

Test Loss: 1.4892, Test Accuracy: 45.40%

Learning Rate: 0.005361

Epoch: 25, Train Loss: 1.4820, Train Accuracy: 46.14%

Epoch: 25, Test Loss: 1.5126, Test Accuracy: 45.46%

End of Epoch 25:

Epoch 25, Updated Noise Multiplier: 1.1482

Privacy Budget: Epsilon = 1.5340

Train Loss: 1.4820, Train Accuracy: 46.14%

Test Loss: 1.5126, Test Accuracy: 45.46%

Learning Rate: 0.005050

Epoch: 26, Train Loss: 1.4866, Train Accuracy: 45.88%

Epoch: 26, Test Loss: 1.4838, Test Accuracy: 46.05%

End of Epoch 26:

Epoch 26, Updated Noise Multiplier: 1.1487

Privacy Budget: Epsilon = 1.5665

Train Loss: 1.4866, Train Accuracy: 45.88%

Test Loss: 1.4838, Test Accuracy: 46.05%

Learning Rate: 0.004739

Epoch: 27, Train Loss: 1.4692, Train Accuracy: 46.65%

Epoch: 27, Test Loss: 1.4921, Test Accuracy: 45.66%

End of Epoch 27:

Epoch 27, Updated Noise Multiplier: 1.1469

Privacy Budget: Epsilon = 1.5985

Train Loss: 1.4692, Train Accuracy: 46.65%

Test Loss: 1.4921, Test Accuracy: 45.66%

Learning Rate: 0.004430

Epoch: 28, Train Loss: 1.4718, Train Accuracy: 46.81%

Epoch: 28, Test Loss: 1.4787, Test Accuracy: 46.73%

End of Epoch 28:

Epoch 28, Updated Noise Multiplier: 1.1472

Privacy Budget: Epsilon = 1.6301

Train Loss: 1.4718, Train Accuracy: 46.81%

Test Loss: 1.4787, Test Accuracy: 46.73%

Learning Rate: 0.004122

Epoch: 29, Train Loss: 1.4737, Train Accuracy: 46.83%

Epoch: 29, Test Loss: 1.4688, Test Accuracy: 46.74%

End of Epoch 29:

Epoch 29, Updated Noise Multiplier: 1.1474

Privacy Budget: Epsilon = 1.6611

Train Loss: 1.4737, Train Accuracy: 46.83%

Test Loss: 1.4688, Test Accuracy: 46.74%

Learning Rate: 0.003819

Epoch: 30, Train Loss: 1.4572, Train Accuracy: 47.49%

Epoch: 30, Test Loss: 1.4697, Test Accuracy: 46.74%

Averaging model parameters...
End of Epoch 30:

Epoch 30, Updated Noise Multiplier: 1.1457

Privacy Budget: Epsilon = 1.6917

Train Loss: 1.4572, Train Accuracy: 47.49%

Test Loss: 1.4697, Test Accuracy: 46.74%

Learning Rate: 0.003520

Epoch: 31, Train Loss: 1.4657, Train Accuracy: 46.99%

Epoch: 31, Test Loss: 1.4685, Test Accuracy: 47.72%

End of Epoch 31:

Epoch 31, Updated Noise Multiplier: 1.1466

Privacy Budget: Epsilon = 1.7220

Train Loss: 1.4657, Train Accuracy: 46.99%

Test Loss: 1.4685, Test Accuracy: 47.72%

Learning Rate: 0.003228

Epoch: 32, Train Loss: 1.4448, Train Accuracy: 48.11%

Epoch: 32, Test Loss: 1.4755, Test Accuracy: 46.95%

End of Epoch 32:

Epoch 32, Updated Noise Multiplier: 1.1445

Privacy Budget: Epsilon = 1.7518

Train Loss: 1.4448, Train Accuracy: 48.11%

Test Loss: 1.4755, Test Accuracy: 46.95%

Learning Rate: 0.002942

Epoch: 33, Train Loss: 1.4597, Train Accuracy: 47.29%

Epoch: 33, Test Loss: 1.4572, Test Accuracy: 47.46%

End of Epoch 33:

Epoch 33, Updated Noise Multiplier: 1.1460

Privacy Budget: Epsilon = 1.7814

Train Loss: 1.4597, Train Accuracy: 47.29%

Test Loss: 1.4572, Test Accuracy: 47.46%

Learning Rate: 0.002665

Epoch: 34, Train Loss: 1.4486, Train Accuracy: 48.04%

Epoch: 34, Test Loss: 1.4675, Test Accuracy: 47.40%

End of Epoch 34:

Epoch 34, Updated Noise Multiplier: 1.1449

Privacy Budget: Epsilon = 1.8104

Train Loss: 1.4486, Train Accuracy: 48.04%

Test Loss: 1.4675, Test Accuracy: 47.40%

Learning Rate: 0.002398

Epoch: 35, Train Loss: 1.4559, Train Accuracy: 47.85%

Epoch: 35, Test Loss: 1.4682, Test Accuracy: 47.04%

End of Epoch 35:

Epoch 35, Updated Noise Multiplier: 1.1456

Privacy Budget: Epsilon = 1.8392

Train Loss: 1.4559, Train Accuracy: 47.85%

Test Loss: 1.4682, Test Accuracy: 47.04%

Learning Rate: 0.002140

Epoch: 36, Train Loss: 1.4577, Train Accuracy: 48.18%

Epoch: 36, Test Loss: 1.4600, Test Accuracy: 47.69%

End of Epoch 36:

Epoch 36, Updated Noise Multiplier: 1.1458

Privacy Budget: Epsilon = 1.8676

Train Loss: 1.4577, Train Accuracy: 48.18%

Test Loss: 1.4600, Test Accuracy: 47.69%

Learning Rate: 0.001895

Epoch: 37, Train Loss: 1.4509, Train Accuracy: 48.21%

Epoch: 37, Test Loss: 1.4713, Test Accuracy: 47.49%

End of Epoch 37:

Epoch 37, Updated Noise Multiplier: 1.1451

Privacy Budget: Epsilon = 1.8956

Train Loss: 1.4509, Train Accuracy: 48.21%

Test Loss: 1.4713, Test Accuracy: 47.49%

Learning Rate: 0.001661

Epoch: 38, Train Loss: 1.4476, Train Accuracy: 48.32%

Epoch: 38, Test Loss: 1.4780, Test Accuracy: 47.85%

End of Epoch 38:

Epoch 38, Updated Noise Multiplier: 1.1448

Privacy Budget: Epsilon = 1.9234

Train Loss: 1.4476, Train Accuracy: 48.32%

Test Loss: 1.4780, Test Accuracy: 47.85%

Learning Rate: 0.001442

Epoch: 39, Train Loss: 1.4463, Train Accuracy: 48.45%

Epoch: 39, Test Loss: 1.4668, Test Accuracy: 48.02%

End of Epoch 39:

Epoch 39, Updated Noise Multiplier: 1.1446

Privacy Budget: Epsilon = 1.9508

Train Loss: 1.4463, Train Accuracy: 48.45%

Test Loss: 1.4668, Test Accuracy: 48.02%

Learning Rate: 0.001236

Epoch: 40, Train Loss: 1.4444, Train Accuracy: 48.27%

Epoch: 40, Test Loss: 1.4593, Test Accuracy: 47.87%

Averaging model parameters...
End of Epoch 40:

Epoch 40, Updated Noise Multiplier: 1.1444

Privacy Budget: Epsilon = 1.9780

Train Loss: 1.4444, Train Accuracy: 48.27%

Test Loss: 1.4593, Test Accuracy: 47.87%

Learning Rate: 0.001045

Epoch: 41, Train Loss: 1.4457, Train Accuracy: 48.78%

Epoch: 41, Test Loss: 1.4608, Test Accuracy: 48.10%

End of Epoch 41:

Epoch 41, Updated Noise Multiplier: 1.1446

Privacy Budget: Epsilon = 2.0049

Train Loss: 1.4457, Train Accuracy: 48.78%

Test Loss: 1.4608, Test Accuracy: 48.10%

Learning Rate: 0.000871

Epoch: 42, Train Loss: 1.4377, Train Accuracy: 48.60%

Epoch: 42, Test Loss: 1.4671, Test Accuracy: 48.29%

End of Epoch 42:

Epoch 42, Updated Noise Multiplier: 1.1438

Privacy Budget: Epsilon = 2.0315

Train Loss: 1.4377, Train Accuracy: 48.60%

Test Loss: 1.4671, Test Accuracy: 48.29%

Learning Rate: 0.000712

Epoch: 43, Train Loss: 1.4347, Train Accuracy: 48.84%

Epoch: 43, Test Loss: 1.4610, Test Accuracy: 48.37%

End of Epoch 43:

Epoch 43, Updated Noise Multiplier: 1.1435

Privacy Budget: Epsilon = 2.0579

Train Loss: 1.4347, Train Accuracy: 48.84%

Test Loss: 1.4610, Test Accuracy: 48.37%

Learning Rate: 0.000571

Epoch: 44, Train Loss: 1.4465, Train Accuracy: 48.41%

Epoch: 44, Test Loss: 1.4665, Test Accuracy: 48.06%

End of Epoch 44:

Epoch 44, Updated Noise Multiplier: 1.1446

Privacy Budget: Epsilon = 2.0840

Train Loss: 1.4465, Train Accuracy: 48.41%

Test Loss: 1.4665, Test Accuracy: 48.06%

Learning Rate: 0.000448

Epoch: 45, Train Loss: 1.4445, Train Accuracy: 48.55%

Epoch: 45, Test Loss: 1.4637, Test Accuracy: 48.16%

End of Epoch 45:

Epoch 45, Updated Noise Multiplier: 1.1445

Privacy Budget: Epsilon = 2.1098

Train Loss: 1.4445, Train Accuracy: 48.55%

Test Loss: 1.4637, Test Accuracy: 48.16%

Learning Rate: 0.000342

Epoch: 46, Train Loss: 1.4348, Train Accuracy: 49.09%

Epoch: 46, Test Loss: 1.4678, Test Accuracy: 48.34%

End of Epoch 46:

Epoch 46, Updated Noise Multiplier: 1.1435

Privacy Budget: Epsilon = 2.1354

Train Loss: 1.4348, Train Accuracy: 49.09%

Test Loss: 1.4678, Test Accuracy: 48.34%

Learning Rate: 0.000256

Epoch: 47, Train Loss: 1.4422, Train Accuracy: 48.59%

Epoch: 47, Test Loss: 1.4601, Test Accuracy: 48.39%

End of Epoch 47:

Epoch 47, Updated Noise Multiplier: 1.1442

Privacy Budget: Epsilon = 2.1608

Train Loss: 1.4422, Train Accuracy: 48.59%

Test Loss: 1.4601, Test Accuracy: 48.39%

Learning Rate: 0.000188

Epoch: 48, Train Loss: 1.4511, Train Accuracy: 48.51%

Epoch: 48, Test Loss: 1.4590, Test Accuracy: 48.40%

End of Epoch 48:

Epoch 48, Updated Noise Multiplier: 1.1451

Privacy Budget: Epsilon = 2.1859

Train Loss: 1.4511, Train Accuracy: 48.51%

Test Loss: 1.4590, Test Accuracy: 48.40%

Learning Rate: 0.000139

Epoch: 49, Train Loss: 1.4484, Train Accuracy: 48.44%

Epoch: 49, Test Loss: 1.4665, Test Accuracy: 48.50%

End of Epoch 49:

Epoch 49, Updated Noise Multiplier: 1.1448

Privacy Budget: Epsilon = 2.2108

Train Loss: 1.4484, Train Accuracy: 48.44%

Test Loss: 1.4665, Test Accuracy: 48.50%

Learning Rate: 0.000110





dynamic noise addition- reverse

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

def update_noise_multiplier(loss, base_noise, max_noise, min_noise, scale_factor):
    """
    Dynamically update noise multiplier based on training loss.

    Args:
    - loss (float): Current training loss.
    - base_noise (float): Base noise multiplier.
    - max_noise (float): Maximum noise multiplier.
    - min_noise (float): Minimum noise multiplier.
    - scale_factor (float): Sensitivity of noise adjustment to loss.

    Returns:
    - float: Updated noise multiplier.
    """
    # Adjust noise dynamically (higher loss -> lower noise, lower loss -> higher noise)
    noise_multiplier = base_noise - scale_factor * loss
    return max(min_noise, min(noise_multiplier, max_noise))  # Clamp to [min_noise, max_noise]



def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)


# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Parameters for dynamic noise adjustment
base_noise = 1.0  # Initial noise multiplier
max_noise = 3.0   # Maximum noise allowed
min_noise = 0.5   # Minimum noise allowed
scale_factor = 0.1  # Scaling factor to adjust sensitivity


# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)
    # Dynamically adjust noise multiplier based on training loss
    new_noise = update_noise_multiplier(
        loss=train_loss,
        base_noise=base_noise,
        max_noise=max_noise,
        min_noise=min_noise,
        scale_factor=scale_factor
    )
    optimizer.noise_multiplier = new_noise  # Update the noise multiplier
    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f"Epoch {epoch}, Updated Noise Multiplier: {new_noise:.4f}\n")

    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')

print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)
import matplotlib.pyplot as plt

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


Epoch: 1, Train Loss: 2.1077, Train Accuracy: 19.90%

Epoch: 1, Test Loss: 1.9668, Test Accuracy: 25.45%

End of Epoch 1:

Epoch 1, Updated Noise Multiplier: 0.7892

Privacy Budget: Epsilon = 0.4951

Train Loss: 2.1077, Train Accuracy: 19.90%

Test Loss: 1.9668, Test Accuracy: 25.45%

Learning Rate: 0.009990

Epoch: 2, Train Loss: 1.9019, Train Accuracy: 28.59%

Epoch: 2, Test Loss: 1.8318, Test Accuracy: 30.41%

End of Epoch 2:

Epoch 2, Updated Noise Multiplier: 0.8098

Privacy Budget: Epsilon = 1.1101

Train Loss: 1.9019, Train Accuracy: 28.59%

Test Loss: 1.8318, Test Accuracy: 30.41%

Learning Rate: 0.009961

Epoch: 3, Train Loss: 1.7953, Train Accuracy: 32.49%

Epoch: 3, Test Loss: 1.7563, Test Accuracy: 35.05%

End of Epoch 3:

Epoch 3, Updated Noise Multiplier: 0.8205

Privacy Budget: Epsilon = 1.2696

Train Loss: 1.7953, Train Accuracy: 32.49%

Test Loss: 1.7563, Test Accuracy: 35.05%

Learning Rate: 0.009912

Epoch: 4, Train Loss: 1.7235, Train Accuracy: 34.78%

Epoch: 4, Test Loss: 1.7207, Test Accuracy: 34.60%

End of Epoch 4:

Epoch 4, Updated Noise Multiplier: 0.8277

Privacy Budget: Epsilon = 1.3881

Train Loss: 1.7235, Train Accuracy: 34.78%

Test Loss: 1.7207, Test Accuracy: 34.60%

Learning Rate: 0.009844

Epoch: 5, Train Loss: 1.6857, Train Accuracy: 36.08%

Epoch: 5, Test Loss: 1.6810, Test Accuracy: 37.01%

End of Epoch 5:

Epoch 5, Updated Noise Multiplier: 0.8314

Privacy Budget: Epsilon = 1.4904

Train Loss: 1.6857, Train Accuracy: 36.08%

Test Loss: 1.6810, Test Accuracy: 37.01%

Learning Rate: 0.009758

Epoch: 6, Train Loss: 1.6658, Train Accuracy: 37.18%

Epoch: 6, Test Loss: 1.6549, Test Accuracy: 37.98%

End of Epoch 6:

Epoch 6, Updated Noise Multiplier: 0.8334

Privacy Budget: Epsilon = 1.5846

Train Loss: 1.6658, Train Accuracy: 37.18%

Test Loss: 1.6549, Test Accuracy: 37.98%

Learning Rate: 0.009652

Epoch: 7, Train Loss: 1.6335, Train Accuracy: 38.65%

Epoch: 7, Test Loss: 1.6479, Test Accuracy: 38.41%

End of Epoch 7:

Epoch 7, Updated Noise Multiplier: 0.8366

Privacy Budget: Epsilon = 1.6739

Train Loss: 1.6335, Train Accuracy: 38.65%

Test Loss: 1.6479, Test Accuracy: 38.41%

Learning Rate: 0.009529

Epoch: 8, Train Loss: 1.6234, Train Accuracy: 39.25%

Epoch: 8, Test Loss: 1.6175, Test Accuracy: 39.30%

End of Epoch 8:

Epoch 8, Updated Noise Multiplier: 0.8377

Privacy Budget: Epsilon = 1.7580

Train Loss: 1.6234, Train Accuracy: 39.25%

Test Loss: 1.6175, Test Accuracy: 39.30%

Learning Rate: 0.009388

Epoch: 9, Train Loss: 1.6078, Train Accuracy: 40.00%

Epoch: 9, Test Loss: 1.5927, Test Accuracy: 40.68%

End of Epoch 9:

Epoch 9, Updated Noise Multiplier: 0.8392

Privacy Budget: Epsilon = 1.8389

Train Loss: 1.6078, Train Accuracy: 40.00%

Test Loss: 1.5927, Test Accuracy: 40.68%

Learning Rate: 0.009229

Epoch: 10, Train Loss: 1.6090, Train Accuracy: 40.40%

Epoch: 10, Test Loss: 1.5983, Test Accuracy: 40.76%

Averaging model parameters...
End of Epoch 10:

Epoch 10, Updated Noise Multiplier: 0.8391

Privacy Budget: Epsilon = 1.9168

Train Loss: 1.6090, Train Accuracy: 40.40%

Test Loss: 1.5983, Test Accuracy: 40.76%

Learning Rate: 0.009055

Epoch: 11, Train Loss: 1.5915, Train Accuracy: 41.56%

Epoch: 11, Test Loss: 1.5742, Test Accuracy: 42.30%

End of Epoch 11:

Epoch 11, Updated Noise Multiplier: 0.8409

Privacy Budget: Epsilon = 1.9926

Train Loss: 1.5915, Train Accuracy: 41.56%

Test Loss: 1.5742, Test Accuracy: 42.30%

Learning Rate: 0.008864

Epoch: 12, Train Loss: 1.5794, Train Accuracy: 42.14%

Epoch: 12, Test Loss: 1.5754, Test Accuracy: 42.23%

End of Epoch 12:

Epoch 12, Updated Noise Multiplier: 0.8421

Privacy Budget: Epsilon = 2.0656

Train Loss: 1.5794, Train Accuracy: 42.14%

Test Loss: 1.5754, Test Accuracy: 42.23%

Learning Rate: 0.008658

Epoch: 13, Train Loss: 1.5583, Train Accuracy: 43.15%

Epoch: 13, Test Loss: 1.5606, Test Accuracy: 43.48%

End of Epoch 13:

Epoch 13, Updated Noise Multiplier: 0.8442

Privacy Budget: Epsilon = 2.1363

Train Loss: 1.5583, Train Accuracy: 43.15%

Test Loss: 1.5606, Test Accuracy: 43.48%

Learning Rate: 0.008439

Epoch: 14, Train Loss: 1.5572, Train Accuracy: 43.13%

Epoch: 14, Test Loss: 1.5447, Test Accuracy: 43.69%

End of Epoch 14:

Epoch 14, Updated Noise Multiplier: 0.8443

Privacy Budget: Epsilon = 2.2045

Train Loss: 1.5572, Train Accuracy: 43.13%

Test Loss: 1.5447, Test Accuracy: 43.69%

Learning Rate: 0.008205

Epoch: 15, Train Loss: 1.5608, Train Accuracy: 43.26%

Epoch: 15, Test Loss: 1.5583, Test Accuracy: 44.35%

End of Epoch 15:

Epoch 15, Updated Noise Multiplier: 0.8439

Privacy Budget: Epsilon = 2.2712

Train Loss: 1.5608, Train Accuracy: 43.26%

Test Loss: 1.5583, Test Accuracy: 44.35%

Learning Rate: 0.007960

Epoch: 16, Train Loss: 1.5282, Train Accuracy: 44.54%

Epoch: 16, Test Loss: 1.5413, Test Accuracy: 44.41%

End of Epoch 16:

Epoch 16, Updated Noise Multiplier: 0.8472

Privacy Budget: Epsilon = 2.3367

Train Loss: 1.5282, Train Accuracy: 44.54%

Test Loss: 1.5413, Test Accuracy: 44.41%

Learning Rate: 0.007702

Epoch: 17, Train Loss: 1.5254, Train Accuracy: 44.70%

Epoch: 17, Test Loss: 1.5262, Test Accuracy: 44.48%

End of Epoch 17:

Epoch 17, Updated Noise Multiplier: 0.8475

Privacy Budget: Epsilon = 2.3997

Train Loss: 1.5254, Train Accuracy: 44.70%

Test Loss: 1.5262, Test Accuracy: 44.48%

Learning Rate: 0.007435

Epoch: 18, Train Loss: 1.5318, Train Accuracy: 44.74%

Epoch: 18, Test Loss: 1.5281, Test Accuracy: 44.74%

End of Epoch 18:

Epoch 18, Updated Noise Multiplier: 0.8468

Privacy Budget: Epsilon = 2.4614

Train Loss: 1.5318, Train Accuracy: 44.74%

Test Loss: 1.5281, Test Accuracy: 44.74%

Learning Rate: 0.007158

Epoch: 19, Train Loss: 1.5252, Train Accuracy: 45.29%

Epoch: 19, Test Loss: 1.5331, Test Accuracy: 45.07%

End of Epoch 19:

Epoch 19, Updated Noise Multiplier: 0.8475

Privacy Budget: Epsilon = 2.5222

Train Loss: 1.5252, Train Accuracy: 45.29%

Test Loss: 1.5331, Test Accuracy: 45.07%

Learning Rate: 0.006872

Epoch: 20, Train Loss: 1.5287, Train Accuracy: 45.15%

Epoch: 20, Test Loss: 1.5188, Test Accuracy: 45.74%

Averaging model parameters...
End of Epoch 20:

Epoch 20, Updated Noise Multiplier: 0.8471

Privacy Budget: Epsilon = 2.5818

Train Loss: 1.5287, Train Accuracy: 45.15%

Test Loss: 1.5188, Test Accuracy: 45.74%

Learning Rate: 0.006580

Epoch: 21, Train Loss: 1.5222, Train Accuracy: 45.58%

Epoch: 21, Test Loss: 1.5271, Test Accuracy: 46.03%

End of Epoch 21:

Epoch 21, Updated Noise Multiplier: 0.8478

Privacy Budget: Epsilon = 2.6406

Train Loss: 1.5222, Train Accuracy: 45.58%

Test Loss: 1.5271, Test Accuracy: 46.03%

Learning Rate: 0.006281

Epoch: 22, Train Loss: 1.5253, Train Accuracy: 45.47%

Epoch: 22, Test Loss: 1.5501, Test Accuracy: 45.29%

End of Epoch 22:

Epoch 22, Updated Noise Multiplier: 0.8475

Privacy Budget: Epsilon = 2.6982

Train Loss: 1.5253, Train Accuracy: 45.47%

Test Loss: 1.5501, Test Accuracy: 45.29%

Learning Rate: 0.005978

Epoch: 23, Train Loss: 1.5162, Train Accuracy: 46.07%

Epoch: 23, Test Loss: 1.5243, Test Accuracy: 46.57%

End of Epoch 23:

Epoch 23, Updated Noise Multiplier: 0.8484

Privacy Budget: Epsilon = 2.7550

Train Loss: 1.5162, Train Accuracy: 46.07%

Test Loss: 1.5243, Test Accuracy: 46.57%

Learning Rate: 0.005670

Epoch: 24, Train Loss: 1.5146, Train Accuracy: 46.19%

Epoch: 24, Test Loss: 1.5207, Test Accuracy: 46.11%

End of Epoch 24:

Epoch 24, Updated Noise Multiplier: 0.8485

Privacy Budget: Epsilon = 2.8107

Train Loss: 1.5146, Train Accuracy: 46.19%

Test Loss: 1.5207, Test Accuracy: 46.11%

Learning Rate: 0.005361

Epoch: 25, Train Loss: 1.5200, Train Accuracy: 46.42%

Epoch: 25, Test Loss: 1.5269, Test Accuracy: 46.41%

End of Epoch 25:

Epoch 25, Updated Noise Multiplier: 0.8480

Privacy Budget: Epsilon = 2.8655

Train Loss: 1.5200, Train Accuracy: 46.42%

Test Loss: 1.5269, Test Accuracy: 46.41%

Learning Rate: 0.005050

Epoch: 26, Train Loss: 1.5097, Train Accuracy: 46.53%

Epoch: 26, Test Loss: 1.5277, Test Accuracy: 46.59%

End of Epoch 26:

Epoch 26, Updated Noise Multiplier: 0.8490

Privacy Budget: Epsilon = 2.9198

Train Loss: 1.5097, Train Accuracy: 46.53%

Test Loss: 1.5277, Test Accuracy: 46.59%

Learning Rate: 0.004739

Epoch: 27, Train Loss: 1.5131, Train Accuracy: 47.03%

Epoch: 27, Test Loss: 1.5260, Test Accuracy: 46.90%

End of Epoch 27:

Epoch 27, Updated Noise Multiplier: 0.8487

Privacy Budget: Epsilon = 2.9730

Train Loss: 1.5131, Train Accuracy: 47.03%

Test Loss: 1.5260, Test Accuracy: 46.90%

Learning Rate: 0.004430

Epoch: 28, Train Loss: 1.5195, Train Accuracy: 46.74%

Epoch: 28, Test Loss: 1.5339, Test Accuracy: 47.19%

End of Epoch 28:

Epoch 28, Updated Noise Multiplier: 0.8481

Privacy Budget: Epsilon = 3.0256

Train Loss: 1.5195, Train Accuracy: 46.74%

Test Loss: 1.5339, Test Accuracy: 47.19%

Learning Rate: 0.004122

Epoch: 29, Train Loss: 1.5165, Train Accuracy: 47.05%

Epoch: 29, Test Loss: 1.5186, Test Accuracy: 47.21%

End of Epoch 29:

Epoch 29, Updated Noise Multiplier: 0.8484

Privacy Budget: Epsilon = 3.0778

Train Loss: 1.5165, Train Accuracy: 47.05%

Test Loss: 1.5186, Test Accuracy: 47.21%

Learning Rate: 0.003819

Epoch: 30, Train Loss: 1.5181, Train Accuracy: 47.01%

Epoch: 30, Test Loss: 1.5275, Test Accuracy: 47.40%

Averaging model parameters...
End of Epoch 30:

Epoch 30, Updated Noise Multiplier: 0.8482

Privacy Budget: Epsilon = 3.1292

Train Loss: 1.5181, Train Accuracy: 47.01%

Test Loss: 1.5275, Test Accuracy: 47.40%

Learning Rate: 0.003520

Epoch: 31, Train Loss: 1.5188, Train Accuracy: 47.12%

Epoch: 31, Test Loss: 1.5267, Test Accuracy: 46.77%

End of Epoch 31:

Epoch 31, Updated Noise Multiplier: 0.8481

Privacy Budget: Epsilon = 3.1801

Train Loss: 1.5188, Train Accuracy: 47.12%

Test Loss: 1.5267, Test Accuracy: 46.77%

Learning Rate: 0.003228

Epoch: 32, Train Loss: 1.5092, Train Accuracy: 47.23%

Epoch: 32, Test Loss: 1.5251, Test Accuracy: 47.46%

End of Epoch 32:

Epoch 32, Updated Noise Multiplier: 0.8491

Privacy Budget: Epsilon = 3.2303

Train Loss: 1.5092, Train Accuracy: 47.23%

Test Loss: 1.5251, Test Accuracy: 47.46%

Learning Rate: 0.002942

Epoch: 33, Train Loss: 1.5122, Train Accuracy: 47.45%

Epoch: 33, Test Loss: 1.5198, Test Accuracy: 47.74%

End of Epoch 33:

Epoch 33, Updated Noise Multiplier: 0.8488

Privacy Budget: Epsilon = 3.2798

Train Loss: 1.5122, Train Accuracy: 47.45%

Test Loss: 1.5198, Test Accuracy: 47.74%

Learning Rate: 0.002665

Epoch: 34, Train Loss: 1.5093, Train Accuracy: 47.85%

Epoch: 34, Test Loss: 1.5255, Test Accuracy: 47.77%

End of Epoch 34:

Epoch 34, Updated Noise Multiplier: 0.8491

Privacy Budget: Epsilon = 3.3288

Train Loss: 1.5093, Train Accuracy: 47.85%

Test Loss: 1.5255, Test Accuracy: 47.77%

Learning Rate: 0.002398

Epoch: 35, Train Loss: 1.4893, Train Accuracy: 48.25%

Epoch: 35, Test Loss: 1.5090, Test Accuracy: 48.29%

End of Epoch 35:

Epoch 35, Updated Noise Multiplier: 0.8511

Privacy Budget: Epsilon = 3.3772

Train Loss: 1.4893, Train Accuracy: 48.25%

Test Loss: 1.5090, Test Accuracy: 48.29%

Learning Rate: 0.002140

Epoch: 36, Train Loss: 1.5011, Train Accuracy: 48.21%

Epoch: 36, Test Loss: 1.5235, Test Accuracy: 48.06%

End of Epoch 36:

Epoch 36, Updated Noise Multiplier: 0.8499

Privacy Budget: Epsilon = 3.4247

Train Loss: 1.5011, Train Accuracy: 48.21%

Test Loss: 1.5235, Test Accuracy: 48.06%

Learning Rate: 0.001895

Epoch: 37, Train Loss: 1.4957, Train Accuracy: 48.28%

Epoch: 37, Test Loss: 1.5530, Test Accuracy: 47.44%

End of Epoch 37:

Epoch 37, Updated Noise Multiplier: 0.8504

Privacy Budget: Epsilon = 3.4719

Train Loss: 1.4957, Train Accuracy: 48.28%

Test Loss: 1.5530, Test Accuracy: 47.44%

Learning Rate: 0.001661

Epoch: 38, Train Loss: 1.5039, Train Accuracy: 48.04%

Epoch: 38, Test Loss: 1.5137, Test Accuracy: 48.23%

End of Epoch 38:

Epoch 38, Updated Noise Multiplier: 0.8496

Privacy Budget: Epsilon = 3.5186

Train Loss: 1.5039, Train Accuracy: 48.04%

Test Loss: 1.5137, Test Accuracy: 48.23%

Learning Rate: 0.001442

Epoch: 39, Train Loss: 1.5128, Train Accuracy: 48.14%

Epoch: 39, Test Loss: 1.5287, Test Accuracy: 47.90%

End of Epoch 39:

Epoch 39, Updated Noise Multiplier: 0.8487

Privacy Budget: Epsilon = 3.5650

Train Loss: 1.5128, Train Accuracy: 48.14%

Test Loss: 1.5287, Test Accuracy: 47.90%

Learning Rate: 0.001236

Epoch: 40, Train Loss: 1.5013, Train Accuracy: 48.38%

Epoch: 40, Test Loss: 1.5294, Test Accuracy: 48.60%

Averaging model parameters...
End of Epoch 40:

Epoch 40, Updated Noise Multiplier: 0.8499

Privacy Budget: Epsilon = 3.6111

Train Loss: 1.5013, Train Accuracy: 48.38%

Test Loss: 1.5294, Test Accuracy: 48.60%

Learning Rate: 0.001045

Epoch: 41, Train Loss: 1.5076, Train Accuracy: 48.18%

Epoch: 41, Test Loss: 1.5162, Test Accuracy: 48.30%

End of Epoch 41:

Epoch 41, Updated Noise Multiplier: 0.8492

Privacy Budget: Epsilon = 3.6566

Train Loss: 1.5076, Train Accuracy: 48.18%

Test Loss: 1.5162, Test Accuracy: 48.30%

Learning Rate: 0.000871

Epoch: 42, Train Loss: 1.4982, Train Accuracy: 48.66%

Epoch: 42, Test Loss: 1.5317, Test Accuracy: 48.38%

End of Epoch 42:

Epoch 42, Updated Noise Multiplier: 0.8502

Privacy Budget: Epsilon = 3.7019

Train Loss: 1.4982, Train Accuracy: 48.66%

Test Loss: 1.5317, Test Accuracy: 48.38%

Learning Rate: 0.000712

Epoch: 43, Train Loss: 1.4863, Train Accuracy: 49.17%

Epoch: 43, Test Loss: 1.5234, Test Accuracy: 48.38%

End of Epoch 43:

Epoch 43, Updated Noise Multiplier: 0.8514

Privacy Budget: Epsilon = 3.7465

Train Loss: 1.4863, Train Accuracy: 49.17%

Test Loss: 1.5234, Test Accuracy: 48.38%

Learning Rate: 0.000571

Epoch: 44, Train Loss: 1.5078, Train Accuracy: 48.35%

Epoch: 44, Test Loss: 1.5277, Test Accuracy: 47.69%

End of Epoch 44:

Epoch 44, Updated Noise Multiplier: 0.8492

Privacy Budget: Epsilon = 3.7904

Train Loss: 1.5078, Train Accuracy: 48.35%

Test Loss: 1.5277, Test Accuracy: 47.69%

Learning Rate: 0.000448

Epoch: 45, Train Loss: 1.5009, Train Accuracy: 48.56%

Epoch: 45, Test Loss: 1.5292, Test Accuracy: 48.25%

End of Epoch 45:

Epoch 45, Updated Noise Multiplier: 0.8499

Privacy Budget: Epsilon = 3.8345

Train Loss: 1.5009, Train Accuracy: 48.56%

Test Loss: 1.5292, Test Accuracy: 48.25%

Learning Rate: 0.000342

Epoch: 46, Train Loss: 1.4961, Train Accuracy: 48.21%

Epoch: 46, Test Loss: 1.5221, Test Accuracy: 48.65%

End of Epoch 46:

Epoch 46, Updated Noise Multiplier: 0.8504

Privacy Budget: Epsilon = 3.8781

Train Loss: 1.4961, Train Accuracy: 48.21%

Test Loss: 1.5221, Test Accuracy: 48.65%

Learning Rate: 0.000256

Epoch: 47, Train Loss: 1.4960, Train Accuracy: 48.87%

Epoch: 47, Test Loss: 1.5271, Test Accuracy: 48.60%

End of Epoch 47:

Epoch 47, Updated Noise Multiplier: 0.8504

Privacy Budget: Epsilon = 3.9212

Train Loss: 1.4960, Train Accuracy: 48.87%

Test Loss: 1.5271, Test Accuracy: 48.60%

Learning Rate: 0.000188

Epoch: 48, Train Loss: 1.5002, Train Accuracy: 48.70%

Epoch: 48, Test Loss: 1.5160, Test Accuracy: 48.26%

End of Epoch 48:

Epoch 48, Updated Noise Multiplier: 0.8500

Privacy Budget: Epsilon = 3.9640

Train Loss: 1.5002, Train Accuracy: 48.70%

Test Loss: 1.5160, Test Accuracy: 48.26%

Learning Rate: 0.000139

Epoch: 49, Train Loss: 1.5064, Train Accuracy: 48.49%

Epoch: 49, Test Loss: 1.5182, Test Accuracy: 48.29%

End of Epoch 49:

Epoch 49, Updated Noise Multiplier: 0.8494

Privacy Budget: Epsilon = 4.0066

Train Loss: 1.5064, Train Accuracy: 48.49%

Test Loss: 1.5182, Test Accuracy: 48.29%

Learning Rate: 0.000110

Epochs:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Train Accuracy:  [19.896936156560795, 28.591400211699387, 32.48509582683151, 34.77720040928516, 36.07834498227625, 37.182897296438924, 38.65348586917527, 39.25278795442651, 39.99599158232288, 40.404729878942405, 41.56471862340059, 42.14395282586609, 43.14565971529965, 43.13473840415505, 43.26076050968195, 44.539369208193456, 44.69583308327832, 44.737792400593825, 45.293293293293296, 45.15046807826751, 45.584687468846575, 45.4732428115016, 46.07355864811133, 46.18564147838998, 46.41645760160562, 46.53278819945486, 47.02715635841608, 46.740728963790296, 47.05034819498919, 47.010875007510364, 47.12166766887152, 47.23023371456986, 47.44806411738905, 47.852772961878, 48.24809750416642, 48.20516911295469, 48.276274123200096, 48.041681638519584, 48.14158939870269, 48.37776483114691, 48.17719796710553, 48.65687282107088, 49.16706330318896, 48.34563578122202, 48.55914408559144, 48.20814133649918, 48.86671618718757, 48.69601879178462, 48.489407799711124]
Test Accuracy:  [25.450721153846153, 30.408653846153847, 35.046073717948715, 34.59535256410256, 37.009214743589745, 37.98076923076923, 38.411458333333336, 39.30288461538461, 40.67508012820513, 40.755208333333336, 42.297676282051285, 42.2275641025641, 43.47956730769231, 43.68990384615385, 44.35096153846154, 44.41105769230769, 44.48116987179487, 44.74158653846154, 45.07211538461539, 45.7431891025641, 46.03365384615385, 45.29246794871795, 46.57451923076923, 46.11378205128205, 46.41426282051282, 46.594551282051285, 46.89503205128205, 47.185496794871796, 47.20552884615385, 47.395833333333336, 46.774839743589745, 47.45592948717949, 47.736378205128204, 47.766426282051285, 48.28725961538461, 48.05689102564103, 47.43589743589744, 48.22716346153846, 47.89663461538461, 48.59775641025641, 48.29727564102564, 48.37740384615385, 48.37740384615385, 47.68629807692308, 48.24719551282051, 48.64783653846154, 48.59775641025641, 48.25721153846154, 48.28725961538461]
Train Loss:  [2.1076787936381804, 1.9018786314206246, 1.7952720397557969, 1.723455936480791, 1.6856605921036159, 1.6658392924528855, 1.6335161288579305, 1.6234411355776666, 1.6077628386326326, 1.6090013180023586, 1.5914724588394165, 1.579412885201283, 1.5583205149723933, 1.5572168729244134, 1.5608264385125576, 1.5281732595883883, 1.5254454930623373, 1.531776931346991, 1.5252344840612166, 1.5286522816389034, 1.522210448827499, 1.525304250839429, 1.516150983174642, 1.5146153388879238, 1.5200155514937181, 1.5097432118195755, 1.5130534269870857, 1.5194646040598552, 1.5164846285795555, 1.5181064049402873, 1.5188043734966181, 1.5091766711993095, 1.5122448921203613, 1.5092860020123995, 1.4893433271310268, 1.5010878355075152, 1.4956645115827902, 1.503944904376299, 1.5127549727757772, 1.5013328509453014, 1.5076176930696537, 1.4981803172673935, 1.486271905899048, 1.5077756105325162, 1.500902644181863, 1.4961287761345887, 1.4959706043585752, 1.5002067804336547, 1.506384230271364]
Test Loss:  [1.9668038349885206, 1.831763628201607, 1.756287776506864, 1.720740461960817, 1.6810028614142003, 1.654901046019334, 1.6478800681921153, 1.6175165634888868, 1.5926921306512294, 1.5983238006249452, 1.574175143853212, 1.5753605854816926, 1.560589998196333, 1.5447040643447485, 1.5582656157322419, 1.5413012963074904, 1.5262092565878844, 1.5280716388653486, 1.5330724869018946, 1.5187860360512366, 1.5270907909442217, 1.5500561518546863, 1.5242790472813141, 1.5206821942940736, 1.5268951257069905, 1.527703731487959, 1.5260052405870879, 1.533893624941508, 1.5186187028884888, 1.5274776373154078, 1.5266909354772322, 1.5250893892386022, 1.5197787223718104, 1.5255334346722333, 1.5089592322325096, 1.5235129717068794, 1.5529693028865716, 1.5137199194003375, 1.5287455014693432, 1.5293903411963048, 1.5161583576446924, 1.5316905700243437, 1.5234004045144105, 1.5276745343819642, 1.529157042503357, 1.5221433211595585, 1.5271409108088567, 1.5160065125196407, 1.5181760176634178]
Epsilon:  [0.49513272854041585, 1.110054830277703, 1.2695761287087877, 1.388142446736472, 1.4903723958712451, 1.5846235200841585, 1.673868117965625, 1.7579608949355627, 1.8389402672266044, 1.9167562331690153, 1.9925719139717228, 2.0655817846168345, 2.1362881232677715, 2.204483120277118, 2.271193173139369, 2.336689157556696, 2.3996571632268364, 2.461380207490421, 2.5222424924059297, 2.5818407319603827, 2.640575792864219, 2.6981671708189707, 2.7549765983446335, 2.810660434483967, 2.8655057160370223, 2.91975554699084, 2.972980001254774, 3.02561199739297, 3.077762203564096, 3.1291904242183217, 3.1800509543230837, 3.230341141515022, 3.279812902581502, 3.32882533394463, 3.377242186495088, 3.424666521532715, 3.4719065563526477, 3.5185551665283694, 3.564958138989702, 3.6111366447637376, 3.656624489691365, 3.701852723845352, 3.7464713008543606, 3.7904462257915417, 3.834535567089714, 3.8781123300090727, 3.9212378778431236, 3.9640243202815486, 4.006574454879168]

In [ ]:
Epochs=  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Train_Accuracy=  [19.896936156560795, 28.591400211699387, 32.48509582683151, 34.77720040928516, 36.07834498227625, 37.182897296438924, 38.65348586917527, 39.25278795442651, 39.99599158232288, 40.404729878942405, 41.56471862340059, 42.14395282586609, 43.14565971529965, 43.13473840415505, 43.26076050968195, 44.539369208193456, 44.69583308327832, 44.737792400593825, 45.293293293293296, 45.15046807826751, 45.584687468846575, 45.4732428115016, 46.07355864811133, 46.18564147838998, 46.41645760160562, 46.53278819945486, 47.02715635841608, 46.740728963790296, 47.05034819498919, 47.010875007510364, 47.12166766887152, 47.23023371456986, 47.44806411738905, 47.852772961878, 48.24809750416642, 48.20516911295469, 48.276274123200096, 48.041681638519584, 48.14158939870269, 48.37776483114691, 48.17719796710553, 48.65687282107088, 49.16706330318896, 48.34563578122202, 48.55914408559144, 48.20814133649918, 48.86671618718757, 48.69601879178462, 48.489407799711124]
Test_Accuracy= [25.450721153846153, 30.408653846153847, 35.046073717948715, 34.59535256410256, 37.009214743589745, 37.98076923076923, 38.411458333333336, 39.30288461538461, 40.67508012820513, 40.755208333333336, 42.297676282051285, 42.2275641025641, 43.47956730769231, 43.68990384615385, 44.35096153846154, 44.41105769230769, 44.48116987179487, 44.74158653846154, 45.07211538461539, 45.7431891025641, 46.03365384615385, 45.29246794871795, 46.57451923076923, 46.11378205128205, 46.41426282051282, 46.594551282051285, 46.89503205128205, 47.185496794871796, 47.20552884615385, 47.395833333333336, 46.774839743589745, 47.45592948717949, 47.736378205128204, 47.766426282051285, 48.28725961538461, 48.05689102564103, 47.43589743589744, 48.22716346153846, 47.89663461538461, 48.59775641025641, 48.29727564102564, 48.37740384615385, 48.37740384615385, 47.68629807692308, 48.24719551282051, 48.64783653846154, 48.59775641025641, 48.25721153846154, 48.28725961538461]
Train_Loss= [2.1076787936381804, 1.9018786314206246, 1.7952720397557969, 1.723455936480791, 1.6856605921036159, 1.6658392924528855, 1.6335161288579305, 1.6234411355776666, 1.6077628386326326, 1.6090013180023586, 1.5914724588394165, 1.579412885201283, 1.5583205149723933, 1.5572168729244134, 1.5608264385125576, 1.5281732595883883, 1.5254454930623373, 1.531776931346991, 1.5252344840612166, 1.5286522816389034, 1.522210448827499, 1.525304250839429, 1.516150983174642, 1.5146153388879238, 1.5200155514937181, 1.5097432118195755, 1.5130534269870857, 1.5194646040598552, 1.5164846285795555, 1.5181064049402873, 1.5188043734966181, 1.5091766711993095, 1.5122448921203613, 1.5092860020123995, 1.4893433271310268, 1.5010878355075152, 1.4956645115827902, 1.503944904376299, 1.5127549727757772, 1.5013328509453014, 1.5076176930696537, 1.4981803172673935, 1.486271905899048, 1.5077756105325162, 1.500902644181863, 1.4961287761345887, 1.4959706043585752, 1.5002067804336547, 1.506384230271364]
Test_Loss= [1.9668038349885206, 1.831763628201607, 1.756287776506864, 1.720740461960817, 1.6810028614142003, 1.654901046019334, 1.6478800681921153, 1.6175165634888868, 1.5926921306512294, 1.5983238006249452, 1.574175143853212, 1.5753605854816926, 1.560589998196333, 1.5447040643447485, 1.5582656157322419, 1.5413012963074904, 1.5262092565878844, 1.5280716388653486, 1.5330724869018946, 1.5187860360512366, 1.5270907909442217, 1.5500561518546863, 1.5242790472813141, 1.5206821942940736, 1.5268951257069905, 1.527703731487959, 1.5260052405870879, 1.533893624941508, 1.5186187028884888, 1.5274776373154078, 1.5266909354772322, 1.5250893892386022, 1.5197787223718104, 1.5255334346722333, 1.5089592322325096, 1.5235129717068794, 1.5529693028865716, 1.5137199194003375, 1.5287455014693432, 1.5293903411963048, 1.5161583576446924, 1.5316905700243437, 1.5234004045144105, 1.5276745343819642, 1.529157042503357, 1.5221433211595585, 1.5271409108088567, 1.5160065125196407, 1.5181760176634178]
Epsilon= [0.49513272854041585, 1.110054830277703, 1.2695761287087877, 1.388142446736472, 1.4903723958712451, 1.5846235200841585, 1.673868117965625, 1.7579608949355627, 1.8389402672266044, 1.9167562331690153, 1.9925719139717228, 2.0655817846168345, 2.1362881232677715, 2.204483120277118, 2.271193173139369, 2.336689157556696, 2.3996571632268364, 2.461380207490421, 2.5222424924059297, 2.5818407319603827, 2.640575792864219, 2.6981671708189707, 2.7549765983446335, 2.810660434483967, 2.8655057160370223, 2.91975554699084, 2.972980001254774, 3.02561199739297, 3.077762203564096, 3.1291904242183217, 3.1800509543230837, 3.230341141515022, 3.279812902581502, 3.32882533394463, 3.377242186495088, 3.424666521532715, 3.4719065563526477, 3.5185551665283694, 3.564958138989702, 3.6111366447637376, 3.656624489691365, 3.701852723845352, 3.7464713008543606, 3.7904462257915417, 3.834535567089714, 3.8781123300090727, 3.9212378778431236, 3.9640243202815486, 4.006574454879168]


# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(Epochs, Train_Accuracy, label='Train Accuracy', color='blue', marker='o')
plt.plot(Epochs, Test_Accuracy, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(Epsilon, Test_Accuracy, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(Epochs, Train_loss, label='Train Loss', color='red', marker='o')
plt.plot(epochs, Test_loss, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


LR=0.01, BATCH SIZE=256 ----Dynamic noise addition as per entropy

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import opacus
from opacus import PrivacyEngine
import torch.nn.functional as F

train_accuracy_vals=[]
test_accuracy_vals=[]
train_loss_vals=[]
test_loss_vals=[]
epochs=[]
epsilon_vals=[]

def compute_entropy(outputs):
    # Apply softmax to get probabilities
    probabilities = F.softmax(outputs, dim=1)
    # Compute entropy
    entropy = -torch.sum(probabilities * torch.log(probabilities + 1e-10), dim=1)  # Avoid log(0)
    return entropy.mean().item()  # Return the average entropy over the batch

def update_noise_multiplier_based_on_entropy(entropy, base_noise, max_noise, min_noise, scale_factor):
    """
    Dynamically update noise multiplier based on entropy of model's predictions.

    Args:
    - entropy (float): The entropy of the model's output.
    - base_noise (float): Base noise multiplier.
    - max_noise (float): Maximum noise multiplier.
    - min_noise (float): Minimum noise multiplier.
    - scale_factor (float): Sensitivity of noise adjustment to entropy.

    Returns:
    - float: Updated noise multiplier.
    """
    # Inverse relationship between entropy and noise multiplier:
    # Higher entropy -> Lower noise, Lower entropy -> Higher noise
    noise_multiplier = base_noise + scale_factor * (1.0 - entropy)  # Inverse scaling
    return max(min_noise, min(noise_multiplier, max_noise))  # Clamp to [min_noise, max_noise]

def group_grad_clipping(model, max_norm):
    """
    Applies group-wise gradient clipping to the model's parameters.

    Args:
    - model: The neural network model.
    - max_norm: Maximum allowed norm for gradients in each group.
    """
    # Define groups (you can customize these)
    param_groups = {
        "conv_layers": [],
        "linear_layers": [],
    }

    # Assign parameters to groups
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if "conv" in name:
            param_groups["conv_layers"].append(param)
        elif "linear" in name:
            param_groups["linear_layers"].append(param)
        else:
            # Add to a default group if needed
            pass

    # Clip gradients for each group
    for group_name, params in param_groups.items():
        if params:
            torch.nn.utils.clip_grad_norm_(params, max_norm)


# Weight Standardization Layer
class WeightStandardization(nn.Module):
    def __init__(self, conv, eps=1e-5):
        super(WeightStandardization, self).__init__()
        self.conv = conv
        self.eps = eps
        self.weight_mean = nn.Parameter(torch.zeros(1, conv.in_channels, 1, 1))
        self.weight_std = nn.Parameter(torch.ones(1, conv.in_channels, 1, 1))

    def forward(self, x):
        weight = self.conv.weight
        weight = (weight - weight.mean(dim=(1, 2, 3), keepdim=True)) / (weight.std(dim=(1, 2, 3), keepdim=True) + self.eps)
        weight = weight * self.weight_std + self.weight_mean
        return F.conv2d(x, weight, self.conv.bias, self.conv.stride, self.conv.padding)

# Wide Basic Block Definition
class WideBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(WideBasicBlock, self).__init__()
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=in_channels)
        self.conv1 = WeightStandardization(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        self.group_norm2 = nn.GroupNorm(num_groups=4, num_channels=out_channels)
        self.conv2 = WeightStandardization(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = self.conv1(F.relu(self.group_norm1(x)))
        out = self.conv2(F.relu(self.group_norm2(out)))
        out += self.shortcut(x)
        return out

# Wide ResNet Model Definition
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes):
        super(WideResNet, self).__init__()
        assert ((depth - 4) % 6 == 0), "Depth should be 6n+4"
        n = (depth - 4) // 6
        k = widen_factor

        nStages = [16, 16*k, 32*k, 64*k]

        self.in_channels = nStages[0]
        self.conv1 = WeightStandardization(nn.Conv2d(3, nStages[0], kernel_size=3, stride=1, padding=1, bias=False))
        self.layer1 = self._wide_layer(WideBasicBlock, nStages[1], n, stride=1)
        self.layer2 = self._wide_layer(WideBasicBlock, nStages[2], n, stride=2)
        self.layer3 = self._wide_layer(WideBasicBlock, nStages[3], n, stride=2)
        self.group_norm1 = nn.GroupNorm(num_groups=4, num_channels=nStages[3])
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.group_norm1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Define Dataset and DataLoader
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, drop_last=True)

# Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WideResNet(depth=16, widen_factor=4, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)

base_noise = 1.0
max_noise = 2.0
min_noise = 0.1
scale_factor = 0.5

# Initialize the PrivacyEngine
privacy_engine = PrivacyEngine()

# Wrap your model, optimizer, and data loader to make them private
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    target_epsilon=3.0,
    target_delta=1e-5,
    epochs=50,
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    #noise_multiplier=1.0,
    max_grad_norm=1.0,
)

print(f"Using sigma={optimizer.noise_multiplier}\n")

# Parameter Averaging Function
def average_model_parameters(models):
    # Assume models is a list of models (e.g., a list of models from different workers)
    state_dict = models[0].state_dict()
    for key in state_dict:
        state_dict[key] = torch.stack([model.state_dict()[key].float() for model in models], dim=0).mean(dim=0)
    return state_dict

# Training and Testing Functions
def train(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Compute entropy and update noise multiplier
        entropy = compute_entropy(outputs)
        optimizer.noise_multiplier = update_noise_multiplier_based_on_entropy(
            entropy, base_noise, max_noise, min_noise, scale_factor
        )

        # Apply grouped gradient clipping
        group_grad_clipping(model, max_norm=1.0)

        # Step with DP-SGD
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy




def test(epoch):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}% \n')
    return avg_loss, accuracy

# Training Loop with Parameter Averaging
for epoch in range(1, 50):
    train_loss, train_accuracy = train(epoch)
    test_loss, test_accuracy = test(epoch)

    # Update the learning rate using the scheduler
    scheduler.step()  # Adjusts the learning rate based on the cosine schedule

    # Average parameters if applicable (for example, after each epoch in distributed setup)
    # This assumes you have multiple models (e.g., in a federated setting) and want to average them.
    if epoch % 10 == 0:  # Every 10 epochs, average parameters (for example)
        print("Averaging model parameters...")
        # For now, we just average a single model's state_dict (this is for illustration).
        # In a real scenario, you'd average across models from different workers.
        model_state_dict = average_model_parameters([model])

        # Load averaged parameters back into the model
        model.load_state_dict(model_state_dict)
    # Query and print the privacy budget spent so far
    epsilon_spent = privacy_engine.get_epsilon(delta=1e-5)
    # Print statistics for the epoch
    epochs.append(epoch)
    train_accuracy_vals.append(train_accuracy)
    test_accuracy_vals.append(test_accuracy)
    train_loss_vals.append(train_loss)
    test_loss_vals.append(test_loss)
    epsilon_vals.append(epsilon_spent)
    print(f'End of Epoch {epoch}: \n')
    print(f'Privacy Budget: Epsilon = {epsilon_spent:.4f} \n')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% \n')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}% \n')
    # Log current learning rate (optional)
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {current_lr:.6f} \n')

import matplotlib.pyplot as plt

print("Epochs: ", epochs)
print("Train Accuracy: ", train_accuracy_vals)
print("Test Accuracy: ", test_accuracy_vals)
print("Train Loss: ", train_loss_vals)
print("Test Loss: ", test_loss_vals)
print("Epsilon: ", epsilon_vals)

# After the training loop, plot the graphs

# 1. Train Accuracy and Test Accuracy vs Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy_vals, label='Train Accuracy', color='blue', marker='o')
plt.plot(epochs, test_accuracy_vals, label='Test Accuracy', color='green', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 2. Epsilon vs Test Accuracy
plt.figure(figsize=(10, 6))
plt.plot(epsilon_vals, test_accuracy_vals, label='Test Accuracy', color='purple', marker='s')
plt.xlabel('Epsilon (Privacy Budget)')
plt.ylabel('Test Accuracy (%)')
plt.title('Epsilon vs Test Accuracy')
plt.grid(True)
plt.show()

# Optional: 3. Train Loss and Test Loss vs Epochs (you can add this to track losses)
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss_vals, label='Train Loss', color='red', marker='o')
plt.plot(epochs, test_loss_vals, label='Test Loss', color='orange', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Test Loss vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


100%|██████████| 170M/170M [00:03<00:00, 49.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


Using sigma=1.22802734375



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.26 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.25 GiB is free. Process 4517 has 13.50 GiB memory in use. Of the allocated memory 11.05 GiB is allocated by PyTorch, and 2.31 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)